# Deferred Revenue Forecast

This jupyter notebook contains the details Adobe's deferred revenue forecast model in a more readable, easy to understand format.

The program is run from python directly (versus using a notebook like we have here) and the details are available on my github page www.github.com/davidjsmith44/Deferred_Revenue_Forecast
Note: You will need me to give you access to this repository as it is not public


Steps to the program
1. Load up all input data
    - billings history
        - Type A
    - FX rates
    - FX_currency map
    - FX forwards
    - bookings data
 
 
2. Process the billings data into a dataframe that includes the BU, currency, period and every type of billings based on it's rebill frequency
 
3. Process the bookings information

4. Forecast the future billings

5. Basic reporting documents

6. Checking for sanity


NOTE: If you plan on using a Jupyter Notebook to run the Deferred Revenue Forecast, you must first download all of the input data to your local directory. I do not yet know how to use a Jupyter Notebook and access data off of the Treasury server.

The input data sits on the Treasury server under Treasury\Financial_Database\Deferred_Revenue\Inputs\Data_YYYY_pMM

There will be 6 files located at this directory that need to be copied to your local drive (preferably where you have installed Anaconda) under a directory called data

## Code Begins here
Below are standard import statements to include all the functionality of numpy, pandas, matplotlib, pickle and a linear regression moodel from sklearn.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

import pickle
from math import ceil
from sklearn.linear_model import LinearRegression
from scipy.interpolate import interp1d, griddata

## Step 1: Processing Base Billings Data

The billings data comes from tableau and is saved in a file "all_billings_inputs.xlsx" with two sheets of information

"base_billings"
Contains the basic information about all of the billings

In [ ]:
df = pd.read_excel('../data/Data_2020_P06/all_billings_inputs.xlsx', sheet_name='base_billings')

###### Changing the column names early since they are inconsistent across other reports

In [ ]:
df.rename(index = str, columns = {'Document Currency': 'curr',
                                 'Enterprise BU Desc': 'BU',
                                 'Invoice Fiscal Year Period Desc': 'period',
                                 'Product Config Type': 'config',
                                 'Rev Rec Category': 'rev_req_type',
                                 'Rule For Bill Date': 'rebill_rule',
                                 'Completed Sales ( DC )': 'DC_amount',
                                 'Completed Sales': 'US_amount'}, inplace=True)

### To inspect the dataframe we just loaded, you can remove the '#' symbol from the rows below. The # symbol makes the line of code a comment (so the program will not read and execute that line of code). Removing the # will make the line executable and display the results

The df.head(5) command will show the first 5 rows of the billings dataframe. To run this, simply remove the # before this line
 
The df.tail(5) command will show the last 5 rows of the billings dataframe. To run this, simply remove the # before this line

The df.sample(5) command will show a random 5 rows of the rec dataframe. To run this, simply remove the # before this line

Note 1: Only one of these commands can be entered in a single code cell at a time. (So if you remove 2 of the #, only one of the commands will be displayed.

Note 2: The 5 can also be changed, but I think it caps out at 100. For example df.head(30) will show the first 30 rows of the billings dataframe.

Note 3: The code must sit on the far leftmost column of the code window. If there is a space it will cause an error.

In [ ]:
#df.head(10)
#df.tail(5)
df.sample(5)

### Filtering out any currency that has  < 10 transactions. 
###### To see the list of currencies and how any times they appear in the billings database remove the # before the print(vc) statement

In [ ]:
# creates a list of the currencies and the number of transactions for each currency
vc = df['curr'].value_counts()
#print(vc)

#Create variable that is true if the number of transaction is greater than 20, false otherwise
keep_these = vc.values > 20
# filtering only currencies that were greater than 20
keep_curr = vc[keep_these]
a = keep_curr.index

#### Just keeping track of the currencies we removed in our model_dict data structure

In [ ]:
remove_these = vc[vc.values <= 20].index
model_dict = {'curr_removed': list(vc[remove_these].index)}
delete_curr = list(remove_these)
print(model_dict)

#### The FX database does not have information on the following currencies
 - AED (United Arab Emirates Dirham)
 - BMD (Bermudan Dollar)
 - MXP (Mexican Peso)
 - TRY (Turkish Lira)
 
 Below we are adding the Turkish Lira to our list of currencies that should be removed from the dataframe

In [ ]:
if 'TRY' not in model_dict['curr_removed']:
    model_dict['curr_removed'].append('TRY')
    delete_curr.append('TRY')
    a = a.drop('TRY')


###### Clearing out the infrequent currencies from our billings data

In [ ]:
df = df[df['curr'].isin(a)]

print('Model dictionary', model_dict)

In [ ]:
print("---Removing infrequent currencies from billings history---")
print('Total number of currencies in the base billings file: ', len(vc))
if len(model_dict['curr_removed'])==0:
    print('No currencies were removed, all contained 10 or more billings')
    print('Currencies in the base billings file')
    for item in a:
        print(a[item], end = " ")
else:
    print('\n Currencies were removed: ', len(model_dict['curr_removed']))

    for item in remove_these:
        print(item, ', ', end="")
        
    print("\n\n{} Remaining currencies: ".format(len(a)))
    for item in a:
        print(item, ', ', end="")

#### Removing any of the values that are zero

In [ ]:
print('This is the length of the dataframe before removing zeros: ', len(df))
df = df[df['DC_amount']!=0]
print('This is the length of the dataframe after removing zeros: ', len(df))

In [ ]:
df.head(10)
#df.tail(10)
#df.sample(10)

#### Clearing out the Non-Revenue billings from the file
 

In [ ]:
df["Sales Type"].value_counts()

In [ ]:
print('Length of the dataframe before removing non-revenue billings: ', len(df))
df = df[df['Sales Type']!='NON-REV']
print('Length of the dataframe after removing non-revenue billings:  ', len(df))


## Grouping the billings by sales type

Grouping the data by the <b> Sales Type </b> field
 - <i>'RECOGNIZED'</i> sales are perpetual and go straight to revenue without hitting deferred 
 - <i>'PRO-SVC-INV'</i> professional services that are invoiced and go to revenue directly when invoiced
 - <i>'DEFERRED'</i> sales that will sit on the balance sheet in deferred revenue and amortize over their life
 
 #### Below we are creating a seperate dataframe for each of the Sales Types


In [ ]:
rec = df[df['Sales Type']=='RECOGNIZED'].copy()
svc = df[df['Sales Type']=='PRO-SVC-INV'].copy()
dfr = df[df['Sales Type']=='DEFERRED'].copy()

print('Total number of billings:              ', len(df))
print("Number of recognized revenue billings: ", len(rec))
print("Number of service invoiced billings:   ", len(svc))
print("Number of deferred revenue billings:   ", len(dfr))

### Recognized Revenue

The rec.head(5) command will show the first 5 elements of the rec dataframe. To run this, simply remove the # before this line
 
The rec.tail(5) command will show the last 5 elements of the rec dataframe. To run this, simply remove the # before this line

The rec.sample(5) command will show the a random 5 elements of the rec dataframe. To run this, simply remove the # before this line

Note: Only one of these commands can be entered in a single code cell at a time. 

Note: The 5 can also be changed, but I think it caps out at 100

In [ ]:
rec.head(5)
#rec.tail(5)
#rec.sample(5)

##### Below we are grouping the rec dataframe by Currency, Business Unit and Period and cleaning up the data we do not need. Since the recognized revenue go directly to revenue, there is no contract that will renew and need to be modeled in the future.

In [ ]:
# testing groupby object
gb_rec = rec.groupby(['curr', 'BU', 'period'], as_index=False).sum()
gb_rec.drop(labels='Subscription Term', axis=1,inplace =True)

In [ ]:
gb_rec.head(10)
#gb_rec.tail(10)
#gb_rec.sample(10)

### Service Billings

##### Below we are grouping the svc dataframe by Currency, Business Unit and Period and cleaning up the data we do not need. Since the service billings go directly to revenue, there is no contract that will renew and need to be modeled in the future.

In [ ]:
gb_svc = svc.groupby(['curr', 'BU', 'period'], as_index=False).sum()
gb_svc.drop(labels='Subscription Term', axis=1,inplace =True)

In [ ]:
gb_svc.head(15)
#gb_svc.tail(5)
#gb_svc.sample(5)

### Deffered Billings

#### Type B Billings 
Type B billings are service agreements that will have invoices submitted before the billings are reclassified to revenue. If no invoices are assigned to the billings, the billings become revenue in 12 months. Since these billings do not have a contract that will renew in the future, there is no need to model a rebillings of these service based billings


In [ ]:
dfr_b = dfr[dfr['rev_req_type']=='B'].copy()
gb_b = dfr_b.groupby(['curr', 'BU', 'period'], as_index=False).sum()
gb_b.drop(labels='Subscription Term', axis=1, inplace=True)

In [ ]:
gb_b.head(20)
#gb_b.tail(25)
#gb_b.sample(5)

#### Type A Billings

These billings are on a billing plan. The product config tells us how long before they renew

 - '3Y' = 36 months
 - '2Y' = 24 months
 - '1Y' = 12 months
 - 'MTHLY' = 1 month
 
NOTE: There are also other fields in the 'Product Configtype ID' field that do not map well to a rebill period.
To fix this, we need to load up a different file and determine the length of the sales contract (type A no config)


In [ ]:
dfr_a = dfr[dfr['rev_req_type']=='A'].copy()

gb_a = dfr_a.groupby(['curr', 'BU', 'period',
                     'config'], as_index=False).sum()
gb_a.drop(labels='Subscription Term', axis=1, inplace = True)

In [ ]:
#gb_a.head(20)
#gb_a.tail(20)
#gb_a.sample(20)

In [ ]:
gb_a['config'].value_counts()

#### Below is just a check to see how large the billing types are across all periods

In [ ]:
gb_a_config = gb_a.groupby(['config'], as_index=False).sum()
gb_a_config

###### These 'OCONS', 'OENSV', 'ONORE' and 'OUNIV' config types are not actual product config IDs so we have to get them from a different data file. We are excluding these types below.

In [ ]:
config_list = ['1Y', '2Y', '3Y', 'MTHLY']
gb_a_config = gb_a[gb_a['config'].isin(config_list)]

###### Grouping by the config type into gb_a_1Y, gb_a_2Y, gb_a_3y, gb_a_1M dataframes


In [ ]:
gb_a_1Y = gb_a_config[gb_a_config['config']=='1Y'].copy()
gb_a_2Y = gb_a_config[gb_a_config['config']=='2Y'].copy()
gb_a_3Y = gb_a_config[gb_a_config['config']=='3Y'].copy()
gb_a_1M = gb_a_config[gb_a_config['config']=='MTHLY'].copy()

In [ ]:
print('this is the lenght of type A 1M billings: ', len(gb_a_1M))
print('this is the lenght of type A 1Y billings: ', len(gb_a_1Y))
print('this is the lenght of type A 2Y billings: ', len(gb_a_2Y))
print('this is the lenght of type A 3Y billings: ', len(gb_a_3Y))

In [ ]:
#gb_a_2Y.head(5)
#gb_a_1M.tail(5)
#gb_a_3Y.sample(5)

#### TYPE D billings
These billings have a field 'Rule For Bill Date' that determines when new billings will occur
 - Monthly:        *{Y1, Y2, Y3, Y5}*
 - Quarterly:      *YQ*
 - Every 4 months: *YT*  --NOTE: There are only 10 of these, so I am treating these as quarterly--
 - Semi-annual:    *YH*
 - Annual:         *{YA, YC}*
 - Every 2 years:  *Y4*
 - Every 3 years:  *Y7*
 
 We also need to track the type D billings that do not have a 'Rule for Bill Date'

In [ ]:
dfr_d = dfr[dfr['rev_req_type']=='D'].copy()

gb_d = dfr_d.groupby(['curr', 'BU', 'period',
                     'rebill_rule'], as_index=False).sum()
gb_d.drop(labels='Subscription Term', axis=1, inplace = True)

In [ ]:
gb_d['rebill_rule'].value_counts()

###### Grouping these by rebill rule and incorporating rebill rules that have the same rebill period

In [ ]:
gb_d_mthly = gb_d[gb_d['rebill_rule'].isin(['Y1', 'Y2', 'Y3', 'YM'])].copy()
gb_d_mthly.drop(labels='rebill_rule', axis=1,inplace=True)
gb_d_mthly = gb_d_mthly.groupby(['curr', 'BU', 'period']).sum()
gb_d_mthly.reset_index(inplace=True)

gb_d_qtrly = gb_d[gb_d['rebill_rule'].isin(['YQ', 'YY', 'YT'])].copy()
gb_d_qtrly.drop(labels='rebill_rule', axis=1,inplace=True)
gb_d_qtrly = gb_d_qtrly.groupby(['curr', 'BU', 'period']).sum()
gb_d_qtrly.reset_index(inplace=True)

gb_d_semi_ann = gb_d[gb_d['rebill_rule']=='YH']

gb_d_annual = gb_d[gb_d['rebill_rule'].isin(['YA', 'YC', 'YX'])].copy()
gb_d_annual.drop(labels='rebill_rule', axis=1,inplace=True)
gb_d_annual = gb_d_annual.groupby(['curr', 'BU', 'period']).sum()
gb_d_annual.reset_index(inplace=True)

gb_d_two_yrs = gb_d[gb_d['rebill_rule']=='Y4']
gb_d_three_yrs = gb_d[gb_d['rebill_rule']=='Y7']

In [ ]:
#gb_d_qtrly.head(10)
#gb_d_annual.tail(10)
#gb_d_three_yrs.head(10)

In [ ]:
print('Length of monthly', len(gb_d_mthly))
print('Length of quarterly', len(gb_d_qtrly))
print('Length of semi ann', len(gb_d_semi_ann))
print('Length of annual', len(gb_d_annual))
print('Length of two years', len(gb_d_two_yrs))
print('Length of three years', len(gb_d_three_yrs))

## Building a single dataframe that incorporates all of this data

- We will have the following descriptive fields
   - Invoicing Fiscal Year-Period
   - Document Currency Billing Amount
   - USD Billing Amount
   - Enterprise BU

- We will have the following fields based on rebilling rule
   - Recognized
   - Service
   - Monthly
   - Quarterly
   - Annual
   - Two Years
   - Three Years

###### Below uses functions to merge a list of dataframes and move billings amounts to the correct category based on rebill frequency and type


In [ ]:
list_df = [gb_rec, gb_svc, gb_b,
           gb_a_1M,    gb_a_1Y,    gb_a_2Y,       gb_a_3Y, 
           gb_d_mthly, gb_d_qtrly, gb_d_semi_ann, gb_d_annual, gb_d_two_yrs, gb_d_three_yrs]

list_columns = ['recognized', 'service', 'deferred_B', 
        'deferred_1M_a', 'deferred_1Y_a', 'deferred_2Y_a', 'deferred_3Y_a',
        'deferred_1M_d', 'deferred_3M_d', 'deferred_6M_d', 'deferred_1Y_d', 'deferred_2Y_d', 'deferred_3Y_d']


In [ ]:
def sum_USD_amt(list_df, list_columns):
    total_US = []
    for df in list_df:
        total_US.append(df['US_amount'].sum())
    total_df = pd.DataFrame(index = list_columns, columns = ['US_amounts'], data=total_US)
    return total_df

In [ ]:
def merge_all_dataframes(list_df, list_columns):
    for i, df in enumerate(list_df):
        #print('This is i:', i)
        #print('referencing the column: ', list_columns[i])

        if i==0:
            df_merged = list_df[0].copy()
            df_merged.rename(index=str, columns={'DC_amount': list_columns[i]+'_DC', 
                                                 'US_amount': list_columns[i]+'_US'}, inplace=True)
        else:
            df_merged = merge_new_dataframe(df_merged, df, list_columns[i])

    return df_merged
    

In [ ]:
def merge_new_dataframe(old_df, new_df, new_column):
    df_merged = pd.merge(old_df, new_df, how='outer', 
                     left_on=['curr', 'BU', 'period'],
                    right_on=['curr', 'BU', 'period'])
    df_merged.rename(index=str, columns={'DC_amount': new_column+'_DC', 'US_amount': new_column+'_US'}, inplace=True)
    
    #need to drop the product configtype id for merges where the new_df is of type A
    config_str = 'config'
    rule_str = 'rebill_rule'
    if config_str in df_merged.columns:
        df_merged.drop(columns=['config'], inplace=True)
    
    if rule_str in df_merged.columns:
        df_merged.drop(columns=['rebill_rule'], inplace=True)
        
    return df_merged

In [ ]:
def clean_df_columns(df):
    
    # clean up NaNs before adding 
    df = df.fillna(value=0)
    
    # DC columns first
    # Monthly
    df['deferred_1M_DC'] = df['deferred_1M_a_DC']+df['deferred_1M_d_DC']
    df.drop(labels=['deferred_1M_a_DC', 'deferred_1M_d_DC'], axis=1, inplace=True)
    
    # Annual
    df['deferred_1Y_DC'] = df['deferred_1Y_a_DC']+df['deferred_1Y_d_DC']
    df.drop(labels=['deferred_1Y_a_DC', 'deferred_1Y_d_DC'], axis=1, inplace=True)
    
    # Two-Year
    df['deferred_2Y_DC'] = df['deferred_2Y_a_DC']+df['deferred_2Y_d_DC']
    df.drop(labels=['deferred_2Y_a_DC', 'deferred_2Y_d_DC'], axis=1, inplace=True)
    
    #Three-Year
    df['deferred_3Y_DC'] = df['deferred_3Y_a_DC']+df['deferred_3Y_d_DC']
    df.drop(labels=['deferred_3Y_a_DC', 'deferred_3Y_d_DC'], axis=1, inplace=True)
    
    # renaming 3M and 6M
    df.rename(index=str, columns = {'deferred_3M_d_DC':'deferred_3M_DC', 
                               'deferred_6M_d_DC': 'deferred_6M_DC'}, inplace=True)

    # US columns
    # Monthly
    df['deferred_1M_US'] = df['deferred_1M_a_US']+df['deferred_1M_d_US']
    df.drop(labels=['deferred_1M_a_US', 'deferred_1M_d_US'], axis=1, inplace=True)
    
    # Annual
    df['deferred_1Y_US'] = df['deferred_1Y_a_US']+df['deferred_1Y_d_US']
    df.drop(labels=['deferred_1Y_a_US', 'deferred_1Y_d_US'], axis=1, inplace=True)
    
    # Two-Year
    df['deferred_2Y_US'] = df['deferred_2Y_a_US']+df['deferred_2Y_d_US']
    df.drop(labels=['deferred_2Y_a_US', 'deferred_2Y_d_US'], axis=1, inplace=True)
    
    # Three-Year
    df['deferred_3Y_US'] = df['deferred_3Y_a_US']+df['deferred_3Y_d_US']
    df.drop(labels=['deferred_3Y_a_US', 'deferred_3Y_d_US'], axis=1, inplace=True)
    
    # renaming 3M and 6M
    df.rename(index=str, columns = {'deferred_3M_d_US':'deferred_3M_US', 
                               'deferred_6M_d_US': 'deferred_6M_US'}, inplace=True)

    
    #cleaning up the longer column names
    df.rename(index=str, columns = {'curr': 'curr',
                               'BU':'BU',
                               'period':'period'}, inplace=True)
    
    return df

##### The code below uses the functions above to merge all of the dataframes and clean up the columns

In [ ]:
df = merge_all_dataframes(list_df, list_columns)

df = clean_df_columns(df)

## I NEED TO CREATE A BETTER PRESENTATION OF THIS CHECK THAT EVERYTHING MATCHES!!!!

## Need to create a summary report with totals coming from every area to make sure the totals I have make sense

In [ ]:
df.sum()

In [ ]:
total_df = sum_USD_amt(list_df, list_columns)
total_df

In [ ]:
total_df.loc['deferred_1M_d']+total_df.loc['deferred_1M_a']

In [ ]:
# Make this a function to be cleaned up somehow
del dfr
del dfr_a
del dfr_b
del dfr_d
del gb_a
del gb_a_1M
del gb_a_1Y
del gb_a_2Y
del gb_a_3Y
del gb_b, 
del gb_d
del gb_svc, gb_rec, gb_d_two_yrs
del gb_d_qtrly, gb_d_semi_ann


# TO BE DONE:

1. Clean up the type F billings (at least check to see if they are necessary)


###### Loading up the Adobe Financial Calendar to get period start and end dates

In [31]:
# loading Adobe financial calendar and calculating period weeks
df_cal = pd.read_excel('../data/old/ADOBE_FINANCIAL_CALENDAR.xlsx', 'ADBE_cal')
df_cal['Period_Weeks'] = (df_cal['Per_End']-df_cal['Per_Start'])/np.timedelta64(1, 'W')
df_cal['Period_Weeks']=df_cal['Period_Weeks'].astype(int)
df_cal['Period_Weeks'] = df_cal['Period_Weeks']+1

In [ ]:
#df_cal.head(5)
#df_cal.sample(5)
df_cal.tail(5)

___
## Type A No Config Type Billings
___

This file contains type A billings that have a revenue contract start date and end date. We need to map these into the terms of our dataframe.

#### Steps
1. Rename the columns
2. This file has entries for pennies. Need to clear out anything less than $10 in absolute value
3. Determine the length of time between start date and end date
4. Group this dataframe by currency, period and BU
5. Merge this final dataframe with the larger dataframe


###### Note: This file contains two different start date and end date columns. At least one of these columns is populated

In [ ]:
df_A = pd.read_excel('../data/Data_2020_P06/all_billings_inputs.xlsx', sheet_name='type_A_no_config')

In [ ]:
df_A.rename(index=str, columns={'Document Currency':'curr', 
                               'Enterprise BU Desc':'BU',
                               'Invoice Fiscal Year Period Desc':'period',
                               'Rev Rec Contract End Date Hdr':'end_date_1',
                               'Rev Rec Contract End Date Item':'end_date_2',
                               'Rev Rec Contract Start Date Hdr': 'start_date_1',
                               'Rev Rec Contract Start Date Item': 'start_date_2',
                               'Completed Sales ( DC )':'DC_amount',
                               'Completed Sales': 'US_amount'
                               }, inplace=True)



In [ ]:
df_A.head(20)
#df_A.sample(5)
#df_A.tail(5)

##### Removing banned currencies
model_dict

In [ ]:
def remove_bad_currencies(df, model_dict):
    this_list = df['curr'].unique().tolist()
    
    for curr in model_dict['curr_removed']:
        if curr in this_list:
            print('need to ban this currency: ', curr)
            df = df[df['curr']!= curr]
    return df


In [ ]:
df_A = remove_bad_currencies(df_A, model_dict)

###### Handling the duplicate dates by taking a max and creating a start_date and end_date fields in pandas datetime format

In [ ]:
df_A['start_date_str'] = df_A[['start_date_1','start_date_2']].max(axis=1).astype(str)
df_A['end_date_str'] = df_A[['end_date_1','end_date_2']].max(axis=1).astype(str)

df_A['start_date'] = pd.to_datetime(df_A['start_date_str'])
df_A['end_date'] = pd.to_datetime(df_A['end_date_str'])

df_A.drop(labels=['end_date_1', 'end_date_2', 'start_date_1', 'start_date_2',
                  'start_date_str', 'end_date_str'], axis=1, inplace=True)

###### Creating a month_interval field that calculates the difference between the start_date and end_date in months. We will map this number of months into a rebilling frequency (this number of months determines when the contract expires and the deferred revenue model assumes that all attribution is accounted for in our net new billings estimates provided by FP&A)

In [ ]:
df_A['month_interval']=(df_A['end_date']-df_A['start_date'])
df_A['months']= (df_A['month_interval']/ np.timedelta64(1,'M')).round(0)

In [ ]:
df_A.head(10)
#df_A.sample(10)
#df_A.tail(10)

##### Mapping the number of months into our common rebill frequencies (monthly, quarterly, semi-annual, annual, 2 years and 3 years)


In [ ]:
list_rebills = [1, 3, 6, 12, 24, 36]
temp_rebill = np.zeros_like(df_A['months'])
for i in range(len(df_A)):
    temp_rebill[i] = min(list_rebills, key=lambda x:abs(x-df_A['months'][i]))
df_A['rebill_months']=temp_rebill

In [ ]:
fig, axs = plt.subplots(1,1, figsize=(14,6))
axs.scatter(df_A['months'], df_A['rebill_months'])
axs.set_ylabel('Rebill Months')
axs.set_xlabel('Number of months between contract start and end dates')
axs.set_title('Type A billings with no config type rebilling mapping')
print_text = 'No'

In [ ]:
df_A.head(10)
#df_A.sample(10)
#df_A.tail(10)

###### Dropping the columns we no longer need

In [ ]:
df_A.drop(columns = ['start_date', 'end_date', 'month_interval', 'months'], axis=1, inplace=True)

###### Grouping the dataframe by rebill_months using a pivot table

In [ ]:
#medals = df.pivot_table('no of medals', ['Year', 'Country'], 'medal')
temp_DC = df_A.pivot_table('DC_amount', ['curr', 'BU', 'period'], 'rebill_months')
temp_US = df_A.pivot_table('US_amount', ['curr', 'BU', 'period'], 'rebill_months')

###### Filling in any zeros that arise if there is no contract on a specific period, currency and BU for a particular rebill period

In [ ]:
temp_DC = temp_DC.fillna(0)
temp_US = temp_DC.fillna(0)

###### Flattening the pivot table back to a normal dataframe and renaming the columns

In [ ]:
temp_flat_DC = pd.DataFrame(temp_DC.to_records())
temp_flat_US = pd.DataFrame(temp_US.to_records())

In [ ]:
temp_flat_DC.rename(index=str, columns={'1.0':'deferred_1M_DC', 
                               '3.0':'deferred_3M_DC',
                               '6.0':'deferred_6M_DC',
                               '12.0':'deferred_1Y_DC',
                               '24.0':'deferred_2Y_DC',
                               '36.0': 'deferred_3Y_DC'}, inplace=True)

temp_flat_US.rename(index=str, columns={'1.0':'deferred_1M_US', 
                               '3.0':'deferred_3M_US',
                               '6.0':'deferred_6M_US',
                               '12.0':'deferred_1Y_US',
                               '24.0':'deferred_2Y_US',
                               '36.0': 'deferred_3Y_US'}, inplace=True)

In [ ]:
temp_flat_DC.head(20)
#temp_flat_US.sample(20)
#temp_flat_DC.tail(20)

###### Quick check that we have not created duplicate column entries (for example two entries for a period with same BU and currency)

In [ ]:
df_test_dup = df.copy()
orig_len = len(df_test_dup)
print("Original Length of the dataframe before duplicate test: ", orig_len)

df_test_dup =df_test_dup.drop_duplicates(subset=['curr', 'BU', 'period'])
print('New length of database after duplicates have been removed: ',len(df_test_dup))

if orig_len!=len(df_test_dup):
    print('We had duplicates in the dataframe! Look into why')

###### Merging the billings dataframe with the temp_flat_DC dataframe and and temp_flat_US dataframe and filling in any blanks with zero

In [ ]:
df_with_A = pd.merge(df, temp_flat_DC, how='outer',
                    left_on= ['curr', 'BU', 'period'],
                    right_on=['curr', 'BU', 'period'], indicator=True, validate='one_to_one')

df_with_A = df_with_A.fillna(pd.Series(0, index=df_with_A.select_dtypes(exclude='category').columns))

In [ ]:
df_with_all = pd.merge(df_with_A, temp_flat_US, how='outer',
                    left_on= ['curr', 'BU', 'period'],
                    right_on=['curr', 'BU', 'period'])

df_with_all = df_with_all.fillna(pd.Series(0, index=df_with_all.select_dtypes(exclude='category').columns))

In [ ]:
#df_with_all.head(10)
#df_with_all.sample(10)
#df_with_all.tail(10)

###### Combining columns form the different data sources (they get merged with different names) and cleaning up the columns

In [ ]:
df_with_all['deferred_1M_DC']= df_with_all['deferred_1M_DC_x']+df_with_all['deferred_1M_DC_y']
df_with_all['deferred_3M_DC']= df_with_all['deferred_3M_DC_x']+df_with_all['deferred_3M_DC_y']
df_with_all['deferred_6M_DC']= df_with_all['deferred_6M_DC_x']+df_with_all['deferred_6M_DC_y']
df_with_all['deferred_1Y_DC']= df_with_all['deferred_1Y_DC_x']+df_with_all['deferred_1Y_DC_y']
df_with_all['deferred_2Y_DC']= df_with_all['deferred_2Y_DC_x']+df_with_all['deferred_2Y_DC_y']
df_with_all['deferred_3Y_DC']= df_with_all['deferred_3Y_DC_x']+df_with_all['deferred_3Y_DC_y']

df_with_all['deferred_1M_US']= df_with_all['deferred_1M_US_x']+df_with_all['deferred_1M_US_y']
df_with_all['deferred_3M_US']= df_with_all['deferred_3M_US_x']+df_with_all['deferred_3M_US_y']
df_with_all['deferred_6M_US']= df_with_all['deferred_6M_US_x']+df_with_all['deferred_6M_US_y']
df_with_all['deferred_1Y_US']= df_with_all['deferred_1Y_US_x']+df_with_all['deferred_1Y_US_y']
df_with_all['deferred_2Y_US']= df_with_all['deferred_2Y_US_x']+df_with_all['deferred_2Y_US_y']
df_with_all['deferred_3Y_US']= df_with_all['deferred_3Y_US_x']+df_with_all['deferred_3Y_US_y']

df_with_all.drop(labels = ['deferred_1M_DC_x','deferred_1M_DC_y',
                        'deferred_3M_DC_x','deferred_3M_DC_y',
                        'deferred_6M_DC_x','deferred_6M_DC_y',
                        'deferred_1Y_DC_x','deferred_1Y_DC_y',
                        'deferred_2Y_DC_x','deferred_2Y_DC_y',
                        'deferred_3Y_DC_x','deferred_3Y_DC_y',
                        'deferred_1M_US_x','deferred_1M_US_y',   
                        'deferred_3M_US_x','deferred_3M_US_y',
                        'deferred_6M_US_x','deferred_6M_US_y',
                        'deferred_1Y_US_x','deferred_1Y_US_y',
                        'deferred_2Y_US_x','deferred_2Y_US_y',
                        'deferred_3Y_US_x','deferred_3Y_US_y'],
                         axis=1, inplace=True)

In [ ]:
#df_with_all.head(5)
#df_with_all.sample(5)
#df_with_all.tail(5)

###### Checking totals to se if they match what we expect

In [ ]:
print('sum of temp flat DC 1M:      ', temp_flat_DC['deferred_1M_DC'].sum())
print('sum of base_df before DC 1M: ', df['deferred_1M_DC'].sum())
print('sum of final DC 1M:          ', df_with_all['deferred_1M_DC'].sum())

a = temp_flat_DC['deferred_1M_DC'].sum()
b = df['deferred_1M_DC'].sum()
c = df_with_all['deferred_1M_DC'].sum()
print(c)
print(a+b)

# TO BE DONE: Create a table that contains the total billings by DC for each dataframe and each step for auditing

 - start with all of the DC
 - then create function that appends and adds rows
 - then do the same for the DC stuff type_A
 - then check the totals


##### Renaming the cleaned billings dataframe as df_billings

In [ ]:
df_billings = df_with_all.copy()

In [ ]:
#df_billings['period'].value_counts()

###### Checking that there are no bilings from future periods in this dataframe. If so, drop them

In [ ]:
#drop_index= df_billings[df_billings['period']=='2020-07'].index
#df_billings.drop(drop_index, inplace=True)

##### Sorting the dataframe and saving this dataframe for use later in a pickle file

In [ ]:
df_billings = df_billings.sort_values(['curr', 'BU', 'period'], ascending = (True, True, True))

with open('../data/processed/all_billings.p', 'wb') as f:
    pickle.dump(df_billings, f)  

#### Below Load up the billings dataframe (for debugging purposes)

In [2]:
#df_billings = pickle.load( open('../data/processed/all_billings.p', 'rb' ))

### Loading All of the other information we need here from excel files
 - currency_map: contain a mapping of currency the majority of our billings in each country
 - FX_data: contains current spot rates, FX forward rates and FX volatilities
 - FX_forward_rates: contains the forward rates used in the FP&A Plan
 - Bookings Forecast: contains the most recent FP&A net new booking forecast (usually only one fiscal year included)

###### Currency Map

In [3]:
df_curr_map = pd.read_excel("../data/Data_2020_P06/currency_map.xlsx", sheet_name="curr_map")
df_curr_map["Country"] = df_curr_map["Country"].str.replace("\(MA\)", "", case=False)
df_curr_map['Country'] = df_curr_map['Country'].str.strip()

##### FX data

In [4]:
df_FX_rates = pd.read_excel('../data/Data_2020_P06/FX_data.xlsx', sheet_name='to_matlab')
df_FX_rates['VOL_3M'] = df_FX_rates['VOL_3M']/100
df_FX_rates['VOL_6M'] = df_FX_rates['VOL_6M']/100
df_FX_rates['VOL_9M'] = df_FX_rates['VOL_9M']/100
df_FX_rates['VOL_1Y'] = df_FX_rates['VOL_1Y']/100

In [5]:
df_FX_rates.head(5)
#df_FX_rates.sample(5)
#df_FX_rates.tail(5)

,DC,Ticker,Spot,FWD_3M,FWD_6M,FWD_9M,FWD_1Y,VOL_3M,VOL_6M,VOL_9M,VOL_1Y
0,ARS,USDARS,68.98350,80.651000,91.571750,104.085600,113.011300,0.25805,0.276550,0.286138,0.295725
1,AUD,AUDUSD,0.69660,0.696592,0.696469,0.696354,0.696128,0.11025,0.108750,0.106463,0.104175
2,BRL,USDBRL,4.96505,4.983550,4.996050,5.006050,5.028784,0.17940,0.167025,0.160562,0.154100
3,CAD,USDCAD,1.34300,1.343012,1.343061,1.342919,1.343325,0.06725,0.066775,0.065862,0.064950
4,CHF,USDCHF,0.96230,0.959683,0.956987,0.953947,0.951229,0.05850,0.061700,0.062300,0.062900


###### FX Forward Rates used in the FP&A Plan

In [6]:
df_FX_fwds = pd.read_excel('../data/Data_2020_P06/FX_forward_rates.xlsx', sheet_name='forward_data', 
                          skiprows = 1, usecols="C,G")

df_FX_fwds.rename(index=str, columns={'Unnamed: 2': 'curr', 'FWD REF':'forward'}, inplace=True)

In [7]:
# Remove the # below to see the entire list of FX_fwds in the plan
df_FX_fwds

,curr,forward
0,AED,3.675481
1,AUD,1.529132
2,BBD,0.000000
3,BMD,1.000000
4,BRL,5.727924
5,CAD,1.391077
6,CHF,0.968247
7,CLP,819.374500
8,COP,3887.357100
9,CNH,7.153242


##### Bookings Forecast

In [8]:
df_bookings = pd.read_excel('../data/Data_2020_P06/2020_bookings_fcst_Q2.xlsx', sheet_name='source')

In [9]:
df_bookings.head(10)
#df_bookings.sample(10)
#df_bookings.tail(10)

,Bookings Hedge,EBU,Internal Segment,PMBU,Geo,Market Area,Market Segment,Booking Type (High),Booking Type (Low),Fiscal Quarter,Plan,FX Conversion,Value
0,Analyst Adjustment - Ent Greenfield,Shared Marketing Cloud (GP),Shared Marketing Cloud (IS),Platform and Other (PMBU),UNKNOWN,UNKNOWN,COMMERCIAL,Net ACV excluding M&S Attrition,Consulting,Q3 2020,QRF - Current,USD Adjusted,-2.033380e+05
1,Analyst Adjustment - Ent Greenfield,Shared Marketing Cloud (GP),Shared Marketing Cloud (IS),Platform and Other (PMBU),UNKNOWN,UNKNOWN,COMMERCIAL,Net ACV excluding M&S Attrition,Consulting,Q4 2020,QRF - Current,USD Adjusted,-1.967787e+05
2,Analyst Adjustment - Ent Named,Commerce (GP),Magento (IS),Magento (PMBU),AMER (G),United States (MA),COMMERCIAL,NaN,Ending Book of Business,Q1 2020,Actuals,USD Adjusted,-2.000000e+07
3,Analyst Adjustment - Ent Named,Commerce (GP),Magento (IS),Magento (PMBU),AMER (G),United States (MA),COMMERCIAL,NaN,Ending Book of Business,Q2 2020,Actuals,USD Adjusted,-2.000000e+07
4,Analyst Adjustment - Ent Named,Commerce (GP),Magento (IS),Magento (PMBU),AMER (G),AMER # (MA),COMMERCIAL,NaN,Ending Book of Business,Q1 2020,Actuals,USD Adjusted,2.000000e+07
5,Analyst Adjustment - Ent Named,Commerce (GP),Magento (IS),Magento (PMBU),AMER (G),AMER # (MA),COMMERCIAL,NaN,Ending Book of Business,Q2 2020,Actuals,USD Adjusted,2.000000e+07
6,Analyst Adjustment - Ent Named,Commerce (GP),Magento (IS),Magento (PMBU),AMER (G),United States (MA),COMMERCIAL,Ending Book of Business,Beginning Book of Business,Q2 2020,Actuals,USD Adjusted,-2.000000e+07
7,Analyst Adjustment - Ent Named,Commerce (GP),Magento (IS),Magento (PMBU),AMER (G),AMER # (MA),COMMERCIAL,Ending Book of Business,Beginning Book of Business,Q2 2020,Actuals,USD Adjusted,2.000000e+07
8,Analyst Adjustment - Ent Named,Commerce (GP),Magento (IS),Magento (PMBU),AMER (G),United States (MA),COMMERCIAL,ASV & Usage Based & Subscription Attrition,ASV,Q1 2020,Actuals,USD Adjusted,-2.000000e+07
9,Analyst Adjustment - Ent Named,Commerce (GP),Magento (IS),Magento (PMBU),AMER (G),AMER # (MA),COMMERCIAL,ASV & Usage Based & Subscription Attrition,ASV,Q1 2020,Actuals,USD Adjusted,2.000000e+07


### Cleaning up the bookings data
##### NOTE: The bookings spreadsheet looks very different for Q2 versus prior quarters!
 - remove odd strings such as '(GP)' from BU, (IS) from Internal Segment, etc
 - dropping columns we do not need
 - renaming columns to better match our data naming convention
 
 NOTE: The '('  and ')' is a special character so we need to precede these with the escape character '\'
 
 NOTE: 2 The columns also have leading or trailing spaces, we need to strip them

In [10]:
df_bookings['EBU'] = df_bookings['EBU'].str.replace(' \(GP\)', '', case=False)
df_bookings['Internal Segment'] = df_bookings['Internal Segment'].str.replace('\(IS\)', '')
df_bookings['PMBU'] = df_bookings['PMBU'].str.replace('\(PMBU\)', '')
df_bookings['Geo'] = df_bookings['Geo'].str.replace('\(G\)', '')
df_bookings['Market Area'] = df_bookings['Market Area'].str.replace('\(MA\)', '')
df_bookings['Booking Type (Low)'] = df_bookings['Booking Type (Low)'].str.replace('\(MA\)', '')

df_bookings['EBU'] = df_bookings['EBU'].str.strip()
df_bookings['Internal Segment'] = df_bookings['Internal Segment'].str.strip()
df_bookings['PMBU'] = df_bookings['PMBU'].str.strip()
df_bookings['Geo'] = df_bookings['Geo'].str.strip()
df_bookings['Market Area'] = df_bookings['Market Area'].str.strip()
df_bookings['Booking Type (Low)'] = df_bookings['Booking Type (Low)'].str.strip()

In [11]:
df_bookings.drop(columns = ['Bookings Hedge', 'Market Segment', 'Booking Type (High)', 'Plan', 'FX Conversion'], inplace = True)

df_bookings.rename(index=str, columns = {'EBU': 'BU', 
                                        'Internal Segment': 'segment',
                                        'PMBU': 'product',
                                        'Geo':'geo',
                                        'Market Area': 'country',
                                        'Booking Type (Low)': 'booking_type',
                                        'Value': 'US_amount',
                                        'Fiscal Quarter': 'Quarter'}, inplace =True)

In [12]:
df_bookings.head(10)
#df_bookings.sample(10)
#df_bookings.tail(10)

,BU,segment,product,geo,country,booking_type,Quarter,US_amount
0,Shared Marketing Cloud,Shared Marketing Cloud,Platform and Other,UNKNOWN,UNKNOWN,Consulting,Q3 2020,-2.033380e+05
1,Shared Marketing Cloud,Shared Marketing Cloud,Platform and Other,UNKNOWN,UNKNOWN,Consulting,Q4 2020,-1.967787e+05
2,Commerce,Magento,Magento,AMER,United States,Ending Book of Business,Q1 2020,-2.000000e+07
3,Commerce,Magento,Magento,AMER,United States,Ending Book of Business,Q2 2020,-2.000000e+07
4,Commerce,Magento,Magento,AMER,AMER #,Ending Book of Business,Q1 2020,2.000000e+07
5,Commerce,Magento,Magento,AMER,AMER #,Ending Book of Business,Q2 2020,2.000000e+07
6,Commerce,Magento,Magento,AMER,United States,Beginning Book of Business,Q2 2020,-2.000000e+07
7,Commerce,Magento,Magento,AMER,AMER #,Beginning Book of Business,Q2 2020,2.000000e+07
8,Commerce,Magento,Magento,AMER,United States,ASV,Q1 2020,-2.000000e+07
9,Commerce,Magento,Magento,AMER,AMER #,ASV,Q1 2020,2.000000e+07


### There are new BUs now! What the hell!

The pivot table Karen is using only look at 4 EBUs
 - Creative
 - Document Cloud
 - Digital Experience
 - Print & Publishing
 
 The following bookings types are used
 - ASV
 - Total Subscription Attrition
 - Consulting (I do not believe this hits deferred revenue) so we drop this
 
 -NOTE: As per Karen on 6/7/20, we need to add 'Premiere Support' to the ASV totals to get ours to match hers
 
 
#### This is not being done here, we have way too many different items in the 'bookings_type' field

###### The cell below shows samples of what is in the data. Removing one of the parenthesis will execute the code. (One at a time)

In [13]:
df_bookings['BU'].value_counts()
#df_bookings['segment'].value_counts()
#df_bookings['product'].value_counts()
#df_bookings['country'].value_counts()
#df_bookings['booking_type'].value_counts();

Content                        4734
Data & Insights                4651
Document Cloud                 2971
Creative                       2798
Customer Journey Management    2767
Commerce                       1604
Shared Marketing Cloud         1063
Print & Publishing              434
Adobe  Video Solutions            2
AEM Other                         2
Name: BU, dtype: int64

In [14]:
change_list = ['Data & Insights', 
              'Customer Journey Management',
              'Commerce',
              'Content',
              'Shared Marketing Cloud',
              'AEM Other',
              'Adobe  Video Solutions']

len(change_list)

new_BU = ['Experience Cloud']
new_BU_list = new_BU * len(change_list)

change_dict = dict(zip(change_list, new_BU_list))
print(change_dict)
 

{'Data & Insights': 'Experience Cloud', 'Customer Journey Management': 'Experience Cloud', 'Commerce': 'Experience Cloud', 'Content': 'Experience Cloud', 'Shared Marketing Cloud': 'Experience Cloud', 'AEM Other': 'Experience Cloud', 'Adobe  Video Solutions': 'Experience Cloud'}


In [15]:
df_bookings['BU'] = df_bookings['BU'].replace(change_dict)

In [16]:
df_bookings['BU'].value_counts()

Experience Cloud      14823
Document Cloud         2971
Creative               2798
Print & Publishing      434
Name: BU, dtype: int64

In [17]:
#df_bookings['BU'].value_counts()
#df_bookings['segment'].value_counts()
#df_bookings['product'].value_counts()
df_bookings['country'].value_counts()
#df_bookings['booking_type'].value_counts();

United States           2341
Aus and New Zealand     1790
United Kingdom          1395
Germany                 1196
France                  1111
Nordic                  1042
Benelux                 1037
Canada                   994
Southeast Asia           987
Switzerland              955
Japan                    935
India                    931
Italy                    853
Iberica                  698
Hong Kong & Taiwan       631
Brazil                   618
Korea                    615
China                    593
Eastern Europe           419
Strat. Latin America     404
SSA & Israel             352
Middle East              323
Mexico                   298
Russia & CIS             168
UNKNOWN                  160
Mediterranean            127
AMER #                    53
Name: country, dtype: int64

#### The countries now contain two fields that we need to change
- UNKNOWN
- AMER #

These will be changed to United States

In [18]:
df_bookings['country'] = df_bookings['country'].replace({'AMER #': 'United States',
                                                        'UNKNOWN': 'United States'})

In [19]:
df_bookings['country'].value_counts()

United States           2554
Aus and New Zealand     1790
United Kingdom          1395
Germany                 1196
France                  1111
Nordic                  1042
Benelux                 1037
Canada                   994
Southeast Asia           987
Switzerland              955
Japan                    935
India                    931
Italy                    853
Iberica                  698
Hong Kong & Taiwan       631
Brazil                   618
Korea                    615
China                    593
Eastern Europe           419
Strat. Latin America     404
SSA & Israel             352
Middle East              323
Mexico                   298
Russia & CIS             168
Mediterranean            127
Name: country, dtype: int64

In [20]:
df_bookings['booking_type'].value_counts()
#df_bookings.columns

Ending Book of Business         6102
ASV                             3562
Beginning Book of Business      2566
Subscription RBOB               2468
Multi-Year                      2117
Total Subscription Attrition    1927
Other Book of Business          1386
Consulting                       574
Premier Support                  222
Perpetual License                 67
Usage Based Bookings              33
New M&S                            2
Name: booking_type, dtype: int64

##### For the booking_type we need to keep the following fields (and add them)
- ASV
- Total Subscription Attrition
- Premier Support

###### Note: These get summed by their booking amont later in the program, so we don't need to do that here

In [21]:
df_bookings = df_bookings[df_bookings['booking_type'].isin(['ASV', 'Total Subscription Attrition'])]

In [22]:
df_bookings.tail(10)

,BU,segment,product,geo,country,booking_type,Quarter,US_amount
21016,Experience Cloud,Magento,Magento,EMEA,Switzerland,ASV,Q4 2020,-5297.215026
21017,Experience Cloud,Marketo,Marketo,EMEA,Switzerland,ASV,Q4 2020,-1095.741731
21018,Experience Cloud,Magento,Magento,EMEA,Nordic,ASV,Q4 2020,-57076.780633
21019,Experience Cloud,Marketo,Marketo,EMEA,Nordic,ASV,Q4 2020,-11806.470018
21020,Experience Cloud,Magento,Magento,EMEA,United Kingdom,ASV,Q4 2020,-317003.486877
21021,Experience Cloud,Marketo,Marketo,EMEA,United Kingdom,ASV,Q4 2020,-65572.937399
21022,Experience Cloud,Magento,Magento,EMEA,Benelux,ASV,Q4 2020,-2130.334997
21023,Experience Cloud,Marketo,Marketo,EMEA,Benelux,ASV,Q4 2020,-440.664943
21024,Experience Cloud,Magento,Magento,EMEA,France,ASV,Q4 2020,-55351.966795
21025,Experience Cloud,Marketo,Marketo,EMEA,France,ASV,Q4 2020,-11449.688107


##
### Merging the bookings country data to a currency using the currency map dataframe (df_curr_map)

In [23]:
df_curr_map

,Country,Currency
0,Aus and New Zealand,AUD
1,Benelux,EUR
2,Brazil,USD
3,Canada,USD
4,China,USD
5,Eastern Europe,EUR
6,France,EUR
7,Germany,EUR
8,Hong Kong & Taiwan,USD
9,Iberica,EUR


In [24]:
list_book_ctry = df_bookings['country'].unique()
print('Countries in the bookings file: \n', list_book_ctry)

list_curr_map = df_curr_map['Country'].unique()
print('Countries in the currency map file: \n', list_curr_map)


Countries in the bookings file: 
 ['United States' 'Benelux' 'Canada' 'Brazil' 'Mexico'
 'Strat. Latin America' 'Germany' 'Switzerland' 'Nordic' 'United Kingdom'
 'SSA & Israel' 'France' 'Iberica' 'Italy' 'Japan' 'Eastern Europe'
 'Middle East' 'Aus and New Zealand' 'India' 'China' 'Hong Kong & Taiwan'
 'Korea' 'Southeast Asia' 'Russia & CIS' 'Mediterranean']
Countries in the currency map file: 
 ['Aus and New Zealand' 'Benelux' 'Brazil' 'Canada' 'China'
 'Eastern Europe' 'France' 'Germany' 'Hong Kong & Taiwan' 'Iberica'
 'India' 'Italy' 'Japan' 'Korea' 'Mediterranean' 'Mexico' 'Middle East'
 'Nordic' 'Russia & CIS' 'Southeast Asia' 'SSA & Israel'
 'Strat. Latin America' 'Switzerland' 'Taiwan' 'United Kingdom'
 'United States']


##### Checking that we have the currency mapping for every country where we have a bookings forecast

In [25]:
a = list(set(list_book_ctry) & set(list_curr_map))

not_in_map = set(list_book_ctry).difference(set(list_curr_map))
if len(not_in_map)!=0:
    print('There is a bookings currency that is not in the currency map!\nWe need to look into the currency map file and add this!')
else:
    print('The bookings currencies are in the currency map. OK to merge the dataframes.')

The bookings currencies are in the currency map. OK to merge the dataframes.


###### Merge the bookings forecast with the currency map

In [26]:
df_bookings = pd.merge(df_bookings, df_curr_map, how='left', left_on='country', right_on='Country')
# the country and Country are the same so we are dropping one of them
df_bookings = df_bookings.drop('Country', axis=1)

In [27]:
#df_bookings.head(10)
df_bookings.sample(10)
#df_bookings.tail(10)

,BU,segment,product,geo,country,booking_type,Quarter,US_amount,Currency
1194,Experience Cloud,Adobe Experience Manager,Sites,ASIA,Southeast Asia,Total Subscription Attrition,Q4 2020,-7093.127890,USD
2625,Experience Cloud,Adobe Target,Adobe Target,ASIA,Hong Kong & Taiwan,ASV,Q4 2020,11542.248000,USD
5147,Experience Cloud,Magento,Magento,EMEA,Benelux,Total Subscription Attrition,Q3 2020,-136800.270906,EUR
1014,Experience Cloud,Audience Manager,Real Time CDP (IS),EMEA,Benelux,ASV,Q2 2020,78831.797998,EUR
2945,Experience Cloud,Adobe Experience Manager,Sites,AMER,Canada,Total Subscription Attrition,Q1 2020,-173475.277434,USD
2846,Experience Cloud,Magento,Magento,EMEA,Italy,ASV,Q4 2020,21766.175432,EUR
3696,Document Cloud,Acrobat DC,Acrobat Desk,AMER,United States,Total Subscription Attrition,Q1 2020,-176728.000000,USD
347,Document Cloud,Acrobat DC,DCE,JPN,Japan,ASV,Q1 2020,47754.783999,JPY
3043,Document Cloud,Acrobat DC,Acrobat Desk,AMER,United States,Total Subscription Attrition,Q1 2020,-244653.010000,USD
521,Experience Cloud,Adobe Experience Manager,Sites,ASIA,China,ASV,Q3 2020,180000.000000,USD


### Adding periods weeks (from the Adobe calendar) to the billings dataframe

In [28]:
#df_cal.head(10)
#df_cal.sample(10)
#df_cal.tail(10)

##### Creating a column in df_cal with year  '-' the last two digits of the per_ticker to match with the billings dataframe

In [32]:
df_cal['p2digit']=df_cal['Period'].astype(str)
df_cal['p2digit']=df_cal['p2digit'].str.zfill(2)

df_cal['period_match']=df_cal['Year'].astype(str) + '-' + df_cal['p2digit'].astype(str)

df_cal.drop(['p2digit'],axis=1, inplace=True)

In [33]:
#df_cal.head(10)
df_cal.sample(10)
#df_cal.tail(10)

,Year,Quarter,Period,Qtr_Ticker,Qtr_Start,Qtr_End,Per_Ticker,Per_Start,Per_End,Period_Weeks,period_match
229,2029,1,2,Q1_29,2028-12-02,2029-03-02,P2_29,2028-12-30,2029-01-26,4,2029-02
218,2028,1,3,Q1_28,2027-12-04,2028-03-03,P3_28,2028-01-29,2028-03-03,5,2028-03
166,2023,4,11,Q4_23,2023-09-02,2023-12-01,P11_23,2023-09-30,2023-10-27,4,2023-11
38,2013,1,3,Q1_13,2012-12-01,2013-03-01,P3_13,2013-01-26,2013-03-01,5,2013-03
33,2012,4,10,Q4_12,2012-09-01,2012-11-30,P10_12,2012-09-01,2012-09-28,4,2012-10
193,2026,1,2,Q1_26,2025-11-29,2026-02-27,P2_26,2025-12-27,2026-01-23,4,2026-02
71,2015,4,12,Q4_15,2015-08-29,2015-11-27,P12_15,2015-10-24,2015-11-27,5,2015-12
185,2025,2,6,Q2_25,2025-03-01,2025-05-30,P6_25,2025-04-26,2025-05-30,5,2025-06
196,2026,2,5,Q2_26,2026-02-28,2026-05-29,P5_26,2026-03-28,2026-04-24,4,2026-05
181,2025,1,2,Q1_25,2024-11-30,2025-02-28,P2_25,2024-12-28,2025-01-24,4,2025-02


##### Getting the calendar ready to be merged with the df_billings dataframe by removing columns that are not needed

In [34]:
df_cal_2_merge = df_cal.copy()
df_cal_2_merge.drop(['Year', 'Quarter', 'Period', 'Qtr_Ticker', 'Qtr_Start', 'Qtr_End', 'Per_Start',
                     'Per_Ticker','Per_End'], axis=1, inplace=True)

##### Merging the calendar periods with the periods in the df_billings dataframe to bring over period weeks

In [35]:
df_billings = df_billings.merge(df_cal_2_merge, how='left', left_on='period', right_on='period_match')

In [36]:
df_billings.columns

Index(['curr', 'BU', 'period', 'recognized_DC', 'recognized_US', 'service_DC',
       'service_US', 'deferred_B_DC', 'deferred_B_US', '_merge',
       'deferred_1M_DC', 'deferred_3M_DC', 'deferred_6M_DC', 'deferred_1Y_DC',
       'deferred_2Y_DC', 'deferred_3Y_DC', 'deferred_1M_US', 'deferred_3M_US',
       'deferred_6M_US', 'deferred_1Y_US', 'deferred_2Y_US', 'deferred_3Y_US',
       'Period_Weeks', 'period_match'],
      dtype='object')

In [37]:
#df_billings.drop(['period_match', '_merge'], axis=1, inplace=True)
df_billings.drop(['period_match'], axis=1, inplace=True)

In [38]:
df_billings.columns

Index(['curr', 'BU', 'period', 'recognized_DC', 'recognized_US', 'service_DC',
       'service_US', 'deferred_B_DC', 'deferred_B_US', '_merge',
       'deferred_1M_DC', 'deferred_3M_DC', 'deferred_6M_DC', 'deferred_1Y_DC',
       'deferred_2Y_DC', 'deferred_3Y_DC', 'deferred_1M_US', 'deferred_3M_US',
       'deferred_6M_US', 'deferred_1Y_US', 'deferred_2Y_US', 'deferred_3Y_US',
       'Period_Weeks'],
      dtype='object')

In [39]:
df_billings.head(5)
#df_billings.sample(5)
#df_billings.tail(5)

,curr,BU,period,recognized_DC,recognized_US,service_DC,service_US,deferred_B_DC,deferred_B_US,_merge,...,deferred_1Y_DC,deferred_2Y_DC,deferred_3Y_DC,deferred_1M_US,deferred_3M_US,deferred_6M_US,deferred_1Y_US,deferred_2Y_US,deferred_3Y_US,Period_Weeks
0,ARS,Creative,2019-03,0.0,0.00,0.0,0.0,0.0,0.0,left_only,...,241380.00,0.0,0.0,10774.56,0.0,0.0,6194.45,0.0,0.0,5
1,ARS,Creative,2019-04,0.0,0.00,0.0,0.0,0.0,0.0,left_only,...,1112854.12,0.0,0.0,46686.26,0.0,0.0,27486.58,0.0,0.0,4
2,ARS,Creative,2019-05,0.0,0.00,0.0,0.0,0.0,0.0,left_only,...,1595808.00,0.0,0.0,75114.30,0.0,0.0,37319.65,0.0,0.0,4
3,ARS,Creative,2019-06,0.0,0.00,0.0,0.0,0.0,0.0,left_only,...,1654238.00,0.0,0.0,128084.89,0.0,0.0,36923.24,0.0,0.0,5
4,ARS,Creative,2019-07,6786.0,155.12,0.0,0.0,0.0,0.0,left_only,...,953875.89,0.0,0.0,146091.12,0.0,0.0,21566.14,0.0,0.0,4


##### Saving these dataframes in as a python dictionary in the pickle file 'all_inputs.p'

In [40]:
df_billings=df_billings.sort_values(['curr', 'BU', 'period'], ascending = (True, True, True))

input_df_dict = {'model_dict': model_dict,
                 'billings':df_billings, 
                 'ADBE_cal':df_cal,
                 'bookings': df_bookings,
                 'FX_forwards': df_FX_fwds,
                 'FX_rates': df_FX_rates
                }

pickle.dump(input_df_dict, open('../data/processed/all_inputs.p', 'wb'))

NameError: name 'model_dict' is not defined

### Cleaning up the billings dataframe
- the billings dataframe does not contain every period if there are no bookings within a period.
- the easiest way to create the forecast requires that we have all of the periods in each BU and currency pair (or at least 36 months worth so that we can incorporate the 3 year deferred bookings

###### The bookings foreacast also contains products such as 'LiveCycle' and 'other solutions' that we do not expect to recieve billings for going forward (there are no booking associated with this) so we need to remove them from the billings data

In [41]:
def add_billings_periods(df_billings):
    # clean up billings by removing LiveCycle and other solutions
    index_lc = df_billings[df_billings['BU']=='LiveCycle'].index
    df_billings.drop(index_lc, inplace=True)

    index_other = df_billings[df_billings['BU']=='Other Solutions'].index
    df_billings.drop(index_other, inplace=True)

    
    all_BU = df_billings['BU'].unique()
    all_curr = df_billings['curr'].unique()

    all_periods = df_billings['period'].unique()
    all_periods = np.sort(all_periods)
    all_periods = all_periods[-36:]


    list_new_BUs = []
    list_new_currs = []
    list_new_periods = []

    for this_BU in all_BU:

        for this_curr in all_curr:

            df_slice = df_billings[(df_billings['BU']== this_BU)&
                                   (df_billings['curr']==this_curr)].copy()

            list_periods = df_slice['period'].unique()
            set_periods = set(list_periods)
            set_all = set(all_periods)

            periods_missing = set_all.difference(set_periods)

            for i in periods_missing:
                list_new_periods.append(i)
                list_new_currs.append(this_curr)
                list_new_BUs.append(this_BU)


    df_to_add = pd.DataFrame({'curr': list_new_currs, 
                              'BU': list_new_BUs,
                              'period': list_new_periods})

    df_billings_check = pd.concat([df_billings, df_to_add], sort=False)

    df_billings_check = df_billings_check.fillna(0)

    df_billings = df_billings_check.copy()
    
    df_billings=df_billings.sort_values(['curr', 'BU', 'period'], ascending = (True, True, True))

    return df_billings


###### Explicit call to the add_billings_periods function is below

In [42]:
print('Length of df_billings before removal of old BUs and adding periods:', len(df_billings))
df_billings = add_billings_periods(df_billings)
print('Length of df_billings after removal of old BUs and adding periods:', len(df_billings))

Length of df_billings before removal of old BUs and adding periods: 3719
Length of df_billings after removal of old BUs and adding periods: 5675


## Cleaning up the bookings dataframe to be incorporated into the deferred model
- The billings dataframe is by period
- the bookings dataframe contains net new bookings by quarter


In [43]:
# find the last period in the billings index
last_period = '2020-06'

list_BUs = df_bookings['BU'].unique()
list_curr = df_bookings['Currency'].unique()

print('This is the list of BUs in the bookings dataframe: ', list_BUs)
print('This is the list of currencies in the bookings dataframe: ', list_curr)


This is the list of BUs in the bookings dataframe:  ['Experience Cloud' 'Creative' 'Document Cloud' 'Print & Publishing']
This is the list of currencies in the bookings dataframe:  ['USD' 'EUR' 'GBP' 'JPY' 'AUD']


##### Creating data to add to the billings dataframe to incorporate period by period billings 
NOTE:  This is just creating the space in the dataframe for the data. We will fill it in later

In [44]:
# creating dataframe of zeros
l_BU = []
l_curr = []
for BU in list_BUs:
    for curr in list_curr:
        l_BU.append(BU)
        l_curr.append(curr)
#print(l_BU)
#print(l_curr)
l_zero = np.zeros(len(l_BU))

In [45]:
data= {'BU':l_BU, 'curr':l_curr, 
      'Q1':l_zero,
      'Q2':l_zero,
      'Q3':l_zero,
      'Q4':l_zero,
      'P01':l_zero,
      'P02':l_zero,
      'P03':l_zero,
      'P04':l_zero,
      'P05':l_zero,
      'P06':l_zero,
      'P07':l_zero,
       'P08':l_zero,
       'P09':l_zero,
       'P10':l_zero,
       'P11':l_zero,
       'P12':l_zero,
      }

df_book_period=pd.DataFrame(data)

In [46]:
df_book_period.head(14)

,BU,curr,Q1,Q2,Q3,Q4,P01,P02,P03,P04,P05,P06,P07,P08,P09,P10,P11,P12
0,Experience Cloud,USD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Experience Cloud,EUR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Experience Cloud,GBP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Experience Cloud,JPY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Experience Cloud,AUD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Creative,USD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Creative,EUR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Creative,GBP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Creative,JPY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Creative,AUD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Uncomment below to remember what the df_bookings looked like

In [47]:
df_bookings.head(10)
#df_bookings.sample(10)
#df_bookings.tail(10)

,BU,segment,product,geo,country,booking_type,Quarter,US_amount,Currency
0,Experience Cloud,Magento,Magento,AMER,United States,ASV,Q1 2020,-2.000000e+07,USD
1,Experience Cloud,Magento,Magento,AMER,United States,ASV,Q1 2020,2.000000e+07,USD
2,Creative,Creative - Professional,Design,EMEA,Benelux,ASV,Q3 2020,1.158456e+05,EUR
3,Creative,Creative - Professional,Design,EMEA,Benelux,ASV,Q4 2020,3.599012e+05,EUR
4,Document Cloud,Adobe Sign,Adobe Sign,EMEA,Benelux,ASV,Q4 2020,9.564208e+04,EUR
5,Document Cloud,Adobe Sign,Adobe Sign,AMER,Canada,Total Subscription Attrition,Q1 2020,-9.028223e+04,USD
6,Document Cloud,Adobe Sign,Adobe Sign,AMER,Canada,Total Subscription Attrition,Q2 2020,-3.342441e+04,USD
7,Document Cloud,Adobe Sign,Adobe Sign,AMER,United States,Total Subscription Attrition,Q1 2020,-9.098347e+05,USD
8,Document Cloud,Adobe Sign,Adobe Sign,AMER,United States,Total Subscription Attrition,Q2 2020,-1.796021e+06,USD
9,Experience Cloud,Marketo,Marketo,AMER,United States,Total Subscription Attrition,Q2 2020,4.148481e+04,USD


##### The cell below fills in the df_book_period dataframe with the quarterly bookings numbers for each BU and currency

In [48]:
# fill in the quarters
for i in range(len(df_book_period['BU'])):
    
    this_BU = df_book_period['BU'][i]
    this_curr = df_book_period['curr'][i]
    this_slice = df_bookings[(df_bookings['BU']==this_BU)&
                          (df_bookings['Currency']==this_curr)]
    
    this_Q1= this_slice[this_slice['Quarter']=='Q1 2020']
    sum_Q1 = this_Q1['US_amount'].sum()
    df_book_period['Q1'].loc[i]=sum_Q1
    
    this_Q2= this_slice[this_slice['Quarter']=='Q2 2020']
    sum_Q2 = this_Q2['US_amount'].sum()
    df_book_period['Q2'].loc[i]=sum_Q2
    
    this_Q3= this_slice[this_slice['Quarter']=='Q4 2020']
    sum_Q3 = this_Q3['US_amount'].sum()
    df_book_period['Q3'].loc[i]=sum_Q3
    
    this_Q4= this_slice[this_slice['Quarter']=='Q4 2020']
    sum_Q4 = this_Q4['US_amount'].sum()
    df_book_period['Q4'].loc[i]=sum_Q4


/Users/davidsmith/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [49]:
df_book_period.head(30)
#df_book_period.sample(10)
#df_book_period.tail(10)

,BU,curr,Q1,Q2,Q3,Q4,P01,P02,P03,P04,P05,P06,P07,P08,P09,P10,P11,P12
0,Experience Cloud,USD,5.089240e+07,5.512803e+07,6.098239e+07,6.098239e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Experience Cloud,EUR,4.025977e+06,8.447370e+06,2.220741e+07,2.220741e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Experience Cloud,GBP,2.846779e+05,3.369984e+06,1.582197e+07,1.582197e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Experience Cloud,JPY,3.406932e+05,-1.059024e+05,6.017759e+06,6.017759e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Experience Cloud,AUD,4.096071e+06,1.039328e+07,7.539179e+06,7.539179e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Creative,USD,3.108810e+07,2.158878e+07,4.771430e+07,4.771430e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Creative,EUR,7.196467e+06,7.136804e+06,1.708626e+07,1.708626e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Creative,GBP,2.163545e+06,1.622120e+06,4.194835e+06,4.194835e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Creative,JPY,2.466702e+06,5.101532e+06,6.046827e+06,6.046827e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Creative,AUD,1.532729e+06,1.636074e+06,3.460680e+06,3.460680e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Creating lists of periods and quarters needed to fill out the df_book_period dataframe

In [50]:
# list of quarters for the percentages

list_q3 = ['2019-07', '2019-08', '2019-09']
list_q4 = ['2019-10', '2019-11', '2019-12']
list_q1 = ['2020-01', '2020-02', '2020-03']
list_q2 = ['2020-04', '2020-05', '2020-06']

list_periods = ['2020-01', '2020-02', '2020-03',
                '2020-04', '2020-05', '2020-06',
                '2019-07', '2019-08', '2019-09',
                '2019-10', '2019-11', '2019-12']

list_p_headers = ['P01', 'P02', 'P03',
                  'P04', 'P05', 'P06',
                  'P07', 'P08', 'P09',
                  'P10', 'P11', 'P12'
                 ]

list_q_headers = ['Q1', 'Q1', 'Q1',
                  'Q2', 'Q2', 'Q2',
                  'Q3', 'Q3', 'Q3',
                  'Q4', 'Q4', 'Q4']


##### adding the booking periods to the dataframe. The bookings are split into periods based on last years percentage of 1 year deferred billings within the quarter.
For example: P1 = 25%, P2 = 30%, P3 = 45% such that the sum is equal to the total quarterly billings last year

In [51]:
for i in range(len(df_book_period['BU'])):
    
    this_BU = df_book_period['BU'][i]
    this_curr = df_book_period['curr'][i]
    
    this_slice = df_billings[(df_billings['BU']==this_BU)&
                          (df_billings['curr']==this_curr)]
    
    for j in range(len(list_periods)):
        this_period = list_periods[j]
        this_header = list_p_headers[j]
        this_quarter = list_q_headers[j]
        this_P_slice = this_slice[this_slice['period']==this_period]
        df_book_period.loc[[i],[this_header]]=this_P_slice['deferred_1Y_DC'].sum()
        
df_book_period['bill_Q1_sum'] = df_book_period['P01'] + df_book_period['P02'] + df_book_period['P03']    
df_book_period['bill_Q2_sum'] = df_book_period['P04'] + df_book_period['P05'] + df_book_period['P06']    
df_book_period['bill_Q3_sum'] = df_book_period['P07'] + df_book_period['P08'] + df_book_period['P09']    
df_book_period['bill_Q4_sum'] = df_book_period['P10'] + df_book_period['P11'] + df_book_period['P12']    

df_book_period['P01'] = df_book_period['Q1']*df_book_period['P01']/df_book_period['bill_Q1_sum']
df_book_period['P02'] = df_book_period['Q1']*df_book_period['P02']/df_book_period['bill_Q1_sum']
df_book_period['P03'] = df_book_period['Q1']*df_book_period['P03']/df_book_period['bill_Q1_sum']

df_book_period['P04'] = df_book_period['Q2']*df_book_period['P04']/df_book_period['bill_Q2_sum']
df_book_period['P05'] = df_book_period['Q2']*df_book_period['P05']/df_book_period['bill_Q2_sum']
df_book_period['P06'] = df_book_period['Q2']*df_book_period['P06']/df_book_period['bill_Q2_sum']

df_book_period['P07'] = df_book_period['Q3']*df_book_period['P07']/df_book_period['bill_Q3_sum']
df_book_period['P08'] = df_book_period['Q3']*df_book_period['P08']/df_book_period['bill_Q3_sum']
df_book_period['P09'] = df_book_period['Q3']*df_book_period['P09']/df_book_period['bill_Q3_sum']

df_book_period['P10'] = df_book_period['Q4']*df_book_period['P10']/df_book_period['bill_Q4_sum']
df_book_period['P11'] = df_book_period['Q4']*df_book_period['P11']/df_book_period['bill_Q4_sum']
df_book_period['P12'] = df_book_period['Q4']*df_book_period['P12']/df_book_period['bill_Q4_sum']
  

In [52]:
#df_book_period.head(10)
#df_book_period.sample(10)
df_book_period.tail(10)

,BU,curr,Q1,Q2,Q3,Q4,P01,P02,P03,P04,...,P07,P08,P09,P10,P11,P12,bill_Q1_sum,bill_Q2_sum,bill_Q3_sum,bill_Q4_sum
10,Document Cloud,USD,1.771112e+07,2.028562e+07,4.705969e+07,4.705969e+07,5.863129e+06,4.485477e+06,7.362509e+06,6.463674e+06,...,1.383350e+07,1.089819e+07,2.232800e+07,1.151129e+07,1.486368e+07,2.068473e+07,1.385831e+08,1.130504e+08,1.316141e+08,1.768703e+08
11,Document Cloud,EUR,2.995354e+06,4.469263e+06,1.591272e+07,1.591272e+07,1.002076e+06,9.866284e+05,1.006649e+06,1.319966e+06,...,4.768523e+06,4.275422e+06,6.868771e+06,2.504676e+06,3.259345e+06,1.014870e+07,3.406740e+07,2.523474e+07,2.601474e+07,4.399013e+07
12,Document Cloud,GBP,2.069625e+06,1.930911e+06,7.655713e+06,7.655713e+06,9.314396e+05,4.637800e+05,6.744052e+05,5.773408e+05,...,2.133806e+06,2.150505e+06,3.371401e+06,2.152079e+06,2.056994e+06,3.446640e+06,1.409661e+07,8.652772e+06,8.716945e+06,1.006637e+07
13,Document Cloud,JPY,1.125937e+06,1.658790e+06,2.764457e+06,2.764457e+06,2.321751e+05,1.877238e+05,7.060379e+05,5.918736e+05,...,7.381935e+05,7.145199e+05,1.311743e+06,6.123127e+05,8.696105e+05,1.282534e+06,1.051697e+09,9.256836e+08,6.578705e+08,9.393522e+08
14,Document Cloud,AUD,8.773844e+05,1.519352e+06,2.210368e+06,2.210368e+06,3.217206e+05,1.806641e+05,3.749998e+05,4.738810e+05,...,6.622612e+05,6.259392e+05,9.221673e+05,5.347804e+05,5.438142e+05,1.131773e+06,1.022950e+07,1.226198e+07,1.167205e+07,1.431008e+07
15,Print & Publishing,USD,1.310532e+06,1.998383e+06,2.379000e+06,2.379000e+06,3.061461e+05,3.446613e+05,6.597246e+05,4.721810e+05,...,6.402181e+05,4.557708e+05,1.283011e+06,7.797855e+05,6.636769e+05,9.355376e+05,2.142521e+07,2.846457e+07,3.185536e+07,3.408717e+07
16,Print & Publishing,EUR,3.103351e+05,1.432046e+05,2.437952e+05,2.437952e+05,1.186513e+05,1.043619e+05,8.732184e+04,5.547398e+04,...,6.304818e+04,6.308583e+04,1.176612e+05,4.482865e+04,5.140373e+04,1.475628e+05,3.461292e+06,3.936604e+06,2.732592e+06,3.146260e+06
17,Print & Publishing,GBP,7.150075e+04,0.000000e+00,0.000000e+00,0.000000e+00,3.190143e+04,1.624073e+04,2.335859e+04,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.781597e+05,1.344354e+06,9.538330e+05,8.068513e+05
18,Print & Publishing,JPY,4.396073e+03,0.000000e+00,0.000000e+00,0.000000e+00,1.767709e+03,1.881718e+03,7.466467e+02,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.897970e+07,2.663410e+07,2.166244e+07,3.727509e+07
19,Print & Publishing,AUD,4.744716e+04,1.700933e+05,4.220965e+04,4.220965e+04,1.211035e+04,1.133869e+04,2.399812e+04,1.523254e+04,...,1.467938e+04,1.306186e+04,1.446841e+04,6.430886e+03,5.646444e+03,3.013232e+04,5.829236e+05,1.139086e+06,5.831776e+05,1.159442e+06


###### Cleaning up the dataframe by dropping the columns we no longer need

In [53]:
df_book_period.drop(['bill_Q1_sum', 'bill_Q2_sum', 'bill_Q3_sum', 'bill_Q4_sum'], axis=1,inplace=True)

In [54]:
df_book_period.columns

Index(['BU', 'curr', 'Q1', 'Q2', 'Q3', 'Q4', 'P01', 'P02', 'P03', 'P04', 'P05',
       'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12'],
      dtype='object')

##### Converting these billings to local currency based on the forward rates at the time the plan was created

In [55]:
df_FX_fwds.set_index('curr', inplace=True)

list_fwds =[]
for i in range(len(df_book_period['curr'])):
    this_curr = df_book_period['curr'][i]
    
    if this_curr == 'USD':
        this_fwd=1
    else:
        this_fwd = df_FX_fwds.loc[this_curr, 'forward']
    
    
    list_fwds.append(this_fwd)
df_book_period['FX_fwd_rate'] = list_fwds

df_book_period['P01_US']=df_book_period['P01']* df_book_period['FX_fwd_rate']
df_book_period['P02_US']=df_book_period['P02']* df_book_period['FX_fwd_rate']
df_book_period['P03_US']=df_book_period['P03']* df_book_period['FX_fwd_rate']
df_book_period['P04_US']=df_book_period['P04']* df_book_period['FX_fwd_rate']
df_book_period['P05_US']=df_book_period['P05']* df_book_period['FX_fwd_rate']
df_book_period['P06_US']=df_book_period['P06']* df_book_period['FX_fwd_rate']
df_book_period['P07_US']=df_book_period['P07']* df_book_period['FX_fwd_rate']
df_book_period['P08_US']=df_book_period['P08']* df_book_period['FX_fwd_rate']
df_book_period['P09_US']=df_book_period['P09']* df_book_period['FX_fwd_rate']
df_book_period['P10_US']=df_book_period['P10']* df_book_period['FX_fwd_rate']
df_book_period['P11_US']=df_book_period['P11']* df_book_period['FX_fwd_rate']
df_book_period['P12_US']=df_book_period['P12']* df_book_period['FX_fwd_rate']

In [56]:
#df_book_period.head(10)
#df_book_period.sample(10)
df_book_period.tail(10)

,BU,curr,Q1,Q2,Q3,Q4,P01,P02,P03,P04,...,P03_US,P04_US,P05_US,P06_US,P07_US,P08_US,P09_US,P10_US,P11_US,P12_US
10,Document Cloud,USD,1.771112e+07,2.028562e+07,4.705969e+07,4.705969e+07,5.863129e+06,4.485477e+06,7.362509e+06,6.463674e+06,...,7.362509e+06,6.463674e+06,6.517373e+06,7.304572e+06,1.383350e+07,1.089819e+07,2.232800e+07,1.151129e+07,1.486368e+07,2.068473e+07
11,Document Cloud,EUR,2.995354e+06,4.469263e+06,1.591272e+07,1.591272e+07,1.002076e+06,9.866284e+05,1.006649e+06,1.319966e+06,...,9.179082e+05,1.203605e+06,1.131415e+06,1.740256e+06,4.348154e+06,3.898522e+06,6.263254e+06,2.283876e+06,2.972018e+06,9.254037e+06
12,Document Cloud,GBP,2.069625e+06,1.930911e+06,7.655713e+06,7.655713e+06,9.314396e+05,4.637800e+05,6.744052e+05,5.773408e+05,...,5.513053e+05,4.719582e+05,4.967519e+05,6.097498e+05,1.744321e+06,1.757971e+06,2.756016e+06,1.759258e+06,1.681529e+06,2.817521e+06
13,Document Cloud,JPY,1.125937e+06,1.658790e+06,2.764457e+06,2.764457e+06,2.321751e+05,1.877238e+05,7.060379e+05,5.918736e+05,...,7.610474e+07,6.379882e+07,4.838280e+07,6.662151e+07,7.957084e+07,7.701903e+07,1.413945e+08,6.600198e+07,9.373644e+07,1.382460e+08
14,Document Cloud,AUD,8.773844e+05,1.519352e+06,2.210368e+06,2.210368e+06,3.217206e+05,1.806641e+05,3.749998e+05,4.738810e+05,...,5.734242e+05,7.246266e+05,5.715926e+05,1.027070e+06,1.012685e+06,9.571436e+05,1.410115e+06,8.177499e+05,8.315636e+05,1.730630e+06
15,Print & Publishing,USD,1.310532e+06,1.998383e+06,2.379000e+06,2.379000e+06,3.061461e+05,3.446613e+05,6.597246e+05,4.721810e+05,...,6.597246e+05,4.721810e+05,6.040821e+05,9.221200e+05,6.402181e+05,4.557708e+05,1.283011e+06,7.797855e+05,6.636769e+05,9.355376e+05
16,Print & Publishing,EUR,3.103351e+05,1.432046e+05,2.437952e+05,2.437952e+05,1.186513e+05,1.043619e+05,8.732184e+04,5.547398e+04,...,7.962399e+04,5.058367e+04,3.978271e+04,4.021407e+04,5.749017e+04,5.752450e+04,1.072887e+05,4.087678e+04,4.687223e+04,1.345544e+05
17,Print & Publishing,GBP,7.150075e+04,0.000000e+00,0.000000e+00,0.000000e+00,3.190143e+04,1.624073e+04,2.335859e+04,0.000000e+00,...,1.909492e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
18,Print & Publishing,JPY,4.396073e+03,0.000000e+00,0.000000e+00,0.000000e+00,1.767709e+03,1.881718e+03,7.466467e+02,0.000000e+00,...,8.048202e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
19,Print & Publishing,AUD,4.744716e+04,1.700933e+05,4.220965e+04,4.220965e+04,1.211035e+04,1.133869e+04,2.399812e+04,1.523254e+04,...,3.669630e+04,2.329257e+04,1.078214e+05,1.289812e+05,2.244671e+04,1.997330e+04,2.212411e+04,9.833674e+03,8.634158e+03,4.607629e+04


##### The df_book_period dataframe now has columns for bookings each period in both local currency and document currency

In [57]:
df_book_period.columns

Index(['BU', 'curr', 'Q1', 'Q2', 'Q3', 'Q4', 'P01', 'P02', 'P03', 'P04', 'P05',
       'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12', 'FX_fwd_rate',
       'P01_US', 'P02_US', 'P03_US', 'P04_US', 'P05_US', 'P06_US', 'P07_US',
       'P08_US', 'P09_US', 'P10_US', 'P11_US', 'P12_US'],
      dtype='object')

## Building the billings forecast in a dataframe called df_fcst

###  Forecasting the billings into the future
#### Steps
 - create list of bill periods that is sorted for the lookup functions
 - create forecast dataframe that includes the same columns (though in document currency) for the billings
 - add the bookings forecast to this data
 - create impact on deferred (project the new waterfall from this_
 - load up accounting's version of the initial waterfall (by BU)
 - reporting

###### creating the list of historical bill periods

In [58]:
list_bill_periods = df_billings['period'].unique()
list_bill_periods.sort()
print(list_bill_periods)

['2016-01' '2016-02' '2016-03' '2016-04' '2016-05' '2016-06' '2016-07'
 '2016-08' '2016-09' '2016-10' '2016-11' '2016-12' '2017-01' '2017-02'
 '2017-03' '2017-04' '2017-05' '2017-06' '2017-07' '2017-08' '2017-09'
 '2017-10' '2017-11' '2017-12' '2018-01' '2018-02' '2018-03' '2018-04'
 '2018-05' '2018-06' '2018-07' '2018-08' '2018-09' '2018-10' '2018-11'
 '2018-12' '2019-01' '2019-02' '2019-03' '2019-04' '2019-05' '2019-06'
 '2019-07' '2019-08' '2019-09' '2019-10' '2019-11' '2019-12' '2020-01'
 '2020-02' '2020-03' '2020-04' '2020-05' '2020-06']


In [59]:
v_BU = df_billings['BU'].copy()
v_curr = df_billings['curr'].copy()
v_both = v_BU + v_curr
v_unique = v_both.unique()

v_un_BU = [sub[:-3] for sub in v_unique]
v_un_curr = [sub[-3:] for sub in v_unique]


In [60]:
list_future_periods = ['2020-07', '2020-08', '2020-09',
                       '2020-10', '2020-11', '2020-12',
                       '2021-01', '2021-02', '2021-03',
                       '2021-04', '2021-05', '2021-06']

In [61]:
# creating the vectors for the future billings dataframe
v_BU_2_df=[]
v_curr_2_df=[]
v_period_2_df = []

for i in range(len(v_un_BU)):
    this_BU = v_un_BU[i]
    this_curr = v_un_curr[i]
    
    for period in list_future_periods:
        v_BU_2_df.append(this_BU)
        v_curr_2_df.append(this_curr)
        v_period_2_df.append(period)

print('This is the length of the vectors: ',len(v_BU_2_df))
    

This is the length of the vectors:  1620


##### Creating a list of the columns that we need to use in the df_billings dataframe (They contain document currency billings)

In [62]:
list_all_columns = df_billings.columns

list_keepers= []
for i in list_all_columns:
    
    if i[-2:]=='DC':
        list_keepers.append(i)

list_keepers

['recognized_DC',
 'service_DC',
 'deferred_B_DC',
 'deferred_1M_DC',
 'deferred_3M_DC',
 'deferred_6M_DC',
 'deferred_1Y_DC',
 'deferred_2Y_DC',
 'deferred_3Y_DC']

##### Creating the df_fcst dataframe with every currency, BU and period we need

In [63]:
df_fcst = pd.DataFrame({'curr': v_curr_2_df,
                        'BU': v_BU_2_df,
                       'period': v_period_2_df})

###### Adding the columns we need to populate (list_keepers)

In [64]:
for col in list_keepers:
    df_fcst[col]=0

In [65]:
df_fcst.head(10)
#df_fcst.sample(10)
#df_fcst.head(10)

,curr,BU,period,recognized_DC,service_DC,deferred_B_DC,deferred_1M_DC,deferred_3M_DC,deferred_6M_DC,deferred_1Y_DC,deferred_2Y_DC,deferred_3Y_DC
0,ARS,Creative,2020-07,0,0,0,0,0,0,0,0,0
1,ARS,Creative,2020-08,0,0,0,0,0,0,0,0,0
2,ARS,Creative,2020-09,0,0,0,0,0,0,0,0,0
3,ARS,Creative,2020-10,0,0,0,0,0,0,0,0,0
4,ARS,Creative,2020-11,0,0,0,0,0,0,0,0,0
5,ARS,Creative,2020-12,0,0,0,0,0,0,0,0,0
6,ARS,Creative,2021-01,0,0,0,0,0,0,0,0,0
7,ARS,Creative,2021-02,0,0,0,0,0,0,0,0,0
8,ARS,Creative,2021-03,0,0,0,0,0,0,0,0,0
9,ARS,Creative,2021-04,0,0,0,0,0,0,0,0,0


##### Adding period weeks to the forecast

In [66]:
df_cal_2_merge = df_cal.copy()
df_cal_2_merge.drop(['Year', 'Quarter', 'Period', 'Qtr_Ticker', 'Qtr_Start', 'Qtr_End', 'Per_Start',
                     'Per_Ticker','Per_End'], axis=1, inplace=True)

df_fcst = df_fcst.merge(df_cal_2_merge, how='left', left_on='period', right_on='period_match')
df_fcst.drop(['period_match'], axis=1, inplace=True)


In [67]:
df_fcst.head(10)
#df_fcst.sample(10)
#df_fcst.tail(10)

,curr,BU,period,recognized_DC,service_DC,deferred_B_DC,deferred_1M_DC,deferred_3M_DC,deferred_6M_DC,deferred_1Y_DC,deferred_2Y_DC,deferred_3Y_DC,Period_Weeks
0,ARS,Creative,2020-07,0,0,0,0,0,0,0,0,0,4
1,ARS,Creative,2020-08,0,0,0,0,0,0,0,0,0,4
2,ARS,Creative,2020-09,0,0,0,0,0,0,0,0,0,5
3,ARS,Creative,2020-10,0,0,0,0,0,0,0,0,0,4
4,ARS,Creative,2020-11,0,0,0,0,0,0,0,0,0,4
5,ARS,Creative,2020-12,0,0,0,0,0,0,0,0,0,5
6,ARS,Creative,2021-01,0,0,0,0,0,0,0,0,0,5
7,ARS,Creative,2021-02,0,0,0,0,0,0,0,0,0,4
8,ARS,Creative,2021-03,0,0,0,0,0,0,0,0,0,5
9,ARS,Creative,2021-04,0,0,0,0,0,0,0,0,0,4


### The functions below create the billings forecast by looking up the historical billings and having them renew
NOTE: The monthly billings are using a linear regression model on the monthly billings / weeks in the month

### TO BE DONE TO COMPLETE:
 - the monthly billings contain several BU, currency pairs that have no monthly billings history. We need to shortcut the program by adding an if statement in that case
 - we need to alter the monthly program to search the periods for the best time period to use (maximizing the R-squared) since some of the BU, currency pairs exhibit growth only after a few years
 - determine which print statement need to be kept to make sure it is running appropriately
 - remove slice error warnings after investigating where the problem occurs

In [68]:
def find_unique_curr_and_BU(df_billings):
    v_BU = df_billings['BU'].copy()
    v_curr = df_billings['curr'].copy()
    v_both = v_BU + v_curr
    v_unique = v_both.unique()

    v_un_BU = [sub[:-3] for sub in v_unique]
    v_un_curr = [sub[-3:] for sub in v_unique]

    return v_un_BU, v_un_curr

In [69]:
def create_billing_forecast(df_billings, df_fcst):

    v_un_BU, v_un_curr = find_unique_curr_and_BU(df_billings)
    
    # new Vectorized approach (sort of)
    counter = 0

    for i in range(len(v_un_BU)):
        this_BU = v_un_BU[i]
        this_curr = v_un_curr[i]
        
        print('working on BU: {0}  and currency: {1}'.format(this_BU, this_curr))
        df_slice = df_billings[(df_billings['BU']==this_BU) &
                                (df_billings['curr']== this_curr)].copy()


        old_per_3Y = list_bill_periods[-36:-24]
        old_per_2Y = list_bill_periods[-24:-12]
        old_per_1Y = list_bill_periods[-12:]
        old_per_6M = list_bill_periods[-6:]
        old_per_3M = list_bill_periods[-3:]

        # three year
        this_v_3yrs = df_slice.loc[df_slice['period'].isin(old_per_3Y), 'deferred_3Y_DC'].copy()
        if len(this_v_3yrs)!=12:
            print(this_BU, this_curr)
            print("There is a period mismatch. length of 3yrs vector = ", len(this_v_3yrs))
            print('Length of df_slice: ', len(df_slice))    
            print('This BU: {0} and this currency: {1}'.format(this_BU, this_curr))

        else:        
            df_fcst.loc[(df_fcst['BU']==this_BU)&
                            (df_fcst['curr']==this_curr),
                            'deferred_3Y_DC'] = this_v_3yrs.values

        #two years
        this_v_2yrs = df_slice.loc[df_slice['period'].isin(old_per_2Y), 'deferred_2Y_DC'].copy()
        if len(this_v_2yrs)!=12:
            print(this_BU, this_curr)
            print("There is a period mismatch. length of 2 yrs vector = ", len(this_v_2yrs))
            print('Length of df_slice: ', len(df_slice))    
            print('This BU: {0} and this currency: {1}'.format(this_BU, this_curr))
        else:
            df_fcst.loc[(df_fcst['BU']==this_BU)&
                        (df_fcst['curr']==this_curr),
                        'deferred_2Y_DC'] = this_v_2yrs.values

        # one year
        this_v_1yrs = df_slice.loc[df_slice['period'].isin(old_per_1Y), 'deferred_1Y_DC'].copy()
        if len(this_v_1yrs)!= 12:
            print(this_BU, this_curr)
            print("There is a period mismatch. length of 1 yr vector = ", len(this_v_1yrs))
            print('Length of df_slice: ', len(df_slice))    

        else:
            df_fcst.loc[(df_fcst['BU']==this_BU)&
                        (df_fcst['curr']==this_curr),
                        'deferred_1Y_DC'] = this_v_1yrs.values

        # six months (we need to append the values to repeat once)
        this_v_6M = df_slice.loc[df_slice['period'].isin(old_per_6M), 'deferred_6M_DC'].copy()
        this_v_6M = this_v_6M.append(this_v_6M, ignore_index=True)

        df_fcst.loc[(df_fcst['BU']==this_BU)&
                    (df_fcst['curr']==this_curr),
                    'deferred_6M_DC'] = this_v_6M.values

        # three months:
        this_v_3M = df_slice.loc[df_slice['period'].isin(old_per_3M), 'deferred_3M_DC'].copy()
        this_v_3M = this_v_3M.append(this_v_3M, ignore_index=True)
        this_v_3M = this_v_3M.append(this_v_3M, ignore_index=True)

        df_fcst.loc[(df_fcst['BU']==this_BU)&
                    (df_fcst['curr']==this_curr),
                    'deferred_3M_DC'] = this_v_3M.values

        # what the hell do we do with the service and recognized revenue billings?
        # RECOGNIZED REVENUE - does not go to deferred, so just take the last 12 month's worth
        this_recog = df_slice.loc[df_slice['period'].isin(old_per_1Y), 'recognized_DC'].copy()
        df_fcst.loc[(df_fcst['BU']==this_BU) &
                    (df_fcst['curr']==this_curr),
                   'recognized_DC'] = this_recog.values

        # SERVICE BASED BILLINGS - for now just use the average of whatever we used last time
        this_svc = df_slice.loc[df_slice['period'].isin(old_per_1Y), 'service_DC'].copy()
        df_fcst.loc[(df_fcst['BU']==this_BU) &
                    (df_fcst['curr']==this_curr),
                   'service_DC'] = this_svc.values

        # Type B Deferred (Service Billings)
        this_type_B = df_slice.loc[df_slice['period'].isin(old_per_1Y), 'deferred_B_DC'].copy()
        df_fcst.loc[(df_fcst['BU']==this_BU) &
                    (df_fcst['curr']==this_curr),
                   'deferred_B_DC'] = this_type_B.values
        
        # MONTHLY BILLINGS
        # here we need to call a seperate function using just the X array that is the one month billings
        this_y= df_slice['deferred_1M_DC'].copy()
        this_y = this_y.to_numpy()
        this_y = this_y.reshape(-1,1)
        
        
        if sum(this_y)!=0:
        
            period_weeks = df_slice['Period_Weeks'].copy()
            period_weeks = period_weeks.to_numpy()
            period_weeks = period_weeks.reshape(-1,1)

            this_y = np.true_divide(this_y, period_weeks)
            this_y = np.nan_to_num(this_y)
            X = np.arange(len(this_y))

            this_model  = build_monthly_forecast(X, this_y)
            weekly_fcst_y = this_model['fcst_y'] 
            
            fcst_slice = df_fcst[(df_fcst['BU']==this_BU)&
                                 (df_fcst['curr']==this_curr)].copy()
            fcst_weeks = fcst_slice['Period_Weeks'].to_numpy()
            fcst_weeks=fcst_weeks.reshape(-1,1)

            period_fcst_y = weekly_fcst_y * fcst_weeks
            
            #print('length of new_y: ', len(fcst_y))
            df_fcst.loc[(df_fcst['BU']==this_BU) &
                        (df_fcst['curr']==this_curr),
                        'deferred_1M_DC'] = period_fcst_y

            df_fcst.loc[(df_fcst['BU']==this_BU)&
                       (df_fcst['curr']==this_curr),
                        'r_squared']= this_model['score']

            df_fcst.loc[(df_fcst['BU']==this_BU)&
                       (df_fcst['curr']==this_curr),
                        'intercept']= this_model['intercept']

            df_fcst.loc[(df_fcst['BU']==this_BU)&
                       (df_fcst['curr']==this_curr),
                        'coeff']= this_model['coeff']

            df_fcst.loc[(df_fcst['BU']==this_BU)&
                   (df_fcst['curr']==this_curr),
                    'X_length']= this_model['first_row']
        
        #print('For this BU: {0} and this currency {1}, we have a score of {2}, and int of {3} and a coeff of {4}'.
        #     format(this_BU, this_curr, this_score, this_int, this_coeff))
        #NOTE: We will need to return two things here
        # First - the df_fcst dataframe
        # second - a dictionary describing the monthly forecasts
    
    
    return df_fcst
    

In [70]:
def build_monthly_forecast(X, y):
    '''
    Need to keep track of the initial X and Y
    Need to track the best X, best Y, best model & best score
    Within the loop, reducing the X and Y to new_x and new_y and keeping track of new model
    If the new model is better, the best_x, best_y, best_model and best_score are overwritten
    
    At the end of the program, the best model is fit and the relevant information is returned
    '''
    
    X = X.reshape(-1,1)
    y = y.reshape(-1,1)
    
    best_X = X.reshape(-1,1)
    best_y = y.reshape(-1,1)
    
    fcst_X = np.arange(np.max(X)+1, np.max(X)+13)
    fcst_X = fcst_X.reshape(-1,1)
    
    # best row tracks the beginning month used for the model
    best_row = 0
    
    # create initial linear regression model, fit it and record score
    best_model = LinearRegression(fit_intercept=True)
    best_model.fit(best_X, best_y)
    best_score = best_model.score(best_X, best_y)
    best_int =   best_model.intercept_
    best_coeff = best_model.coef_

    #print("Model Score :",       best_score)
    #print("Model intercept :",   best_model.intercept_)
    #print("Model Coefficient :", best_model.coef_)

    for start_row in np.arange(1, y.shape[0]-12):
        new_X = X[start_row:]
        new_y = y[start_row:]
        
        new_model = LinearRegression(fit_intercept=True)
        new_model.fit(new_X, new_y)
        new_score = new_model.score(new_X, new_y)
        new_int =   new_model.intercept_
        new_coeff = new_model.coef_
        
        #print("Model Score :",       new_score)
        #print("Model intercept :",   new_model.intercept_)
        #print("Model Coefficient :", new_model.coef_)
        

        # if the new model beats the best model, reassign to the best model
        if new_score > best_score:
            best_model = new_model
            best_score = new_score
            best_X = new_X
            best_y = new_y
            best_row = start_row
            best_int = new_int
            best_coeff = new_coeff
            
    #perform the forecast
    fcst_y = best_model.predict(fcst_X)
    
    
    monthly_model = dict({'model':  best_model,
                     'score':  best_score,
                     'fcst_y': fcst_y,
                     'first_row': best_row,
                     'intercept': best_int,
                     'coeff' : best_coeff
                    })
    
        
    return monthly_model

In [71]:
df_fcst = create_billing_forecast(df_billings, df_fcst)

working on BU: Creative  and currency: ARS
working on BU: DX Other  and currency: ARS
working on BU: Document Cloud  and currency: ARS
working on BU: Experience Cloud  and currency: ARS
working on BU: Print & Publishing  and currency: ARS
working on BU: Creative  and currency: AUD


/Users/davidsmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:109: RuntimeWarning: invalid value encountered in true_divide
/Users/davidsmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:109: RuntimeWarning: invalid value encountered in true_divide


working on BU: DX Other  and currency: AUD
working on BU: Document Cloud  and currency: AUD
working on BU: Experience Cloud  and currency: AUD
working on BU: Print & Publishing  and currency: AUD
working on BU: Creative  and currency: BRL


/Users/davidsmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:109: RuntimeWarning: invalid value encountered in true_divide
/Users/davidsmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:109: RuntimeWarning: invalid value encountered in true_divide


working on BU: DX Other  and currency: BRL
working on BU: Document Cloud  and currency: BRL
working on BU: Experience Cloud  and currency: BRL
working on BU: Print & Publishing  and currency: BRL
working on BU: Creative  and currency: CAD
working on BU: DX Other  and currency: CAD
working on BU: Document Cloud  and currency: CAD
working on BU: Experience Cloud  and currency: CAD


...

working on BU: Document Cloud  and currency: SEK
working on BU: Experience Cloud  and currency: SEK
working on BU: Print & Publishing  and currency: SEK
working on BU: Creative  and currency: SGD
working on BU: DX Other  and currency: SGD
working on BU: Document Cloud  and currency: SGD


/Users/davidsmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:109: RuntimeWarning: invalid value encountered in true_divide
/Users/davidsmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:109: RuntimeWarning: invalid value encountered in true_divide
/Users/davidsmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:109: RuntimeWarning: invalid value encountered in true_divide
/Users/davidsmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:109: RuntimeWarning: invalid value encountered in true_divide


working on BU: Experience Cloud  and currency: SGD
working on BU: Print & Publishing  and currency: SGD
working on BU: Creative  and currency: THB
working on BU: DX Other  and currency: THB
working on BU: Document Cloud  and currency: THB
working on BU: Experience Cloud  and currency: THB
working on BU: Print & Publishing  and currency: THB
working on BU: Creative  and currency: TWD
working on BU: DX Other  and currency: TWD


/Users/davidsmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:109: RuntimeWarning: invalid value encountered in true_divide
/Users/davidsmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:109: RuntimeWarning: invalid value encountered in true_divide


working on BU: Document Cloud  and currency: TWD
working on BU: Experience Cloud  and currency: TWD
working on BU: Print & Publishing  and currency: TWD
working on BU: Creative  and currency: USD
working on BU: DX Other  and currency: USD
working on BU: Document Cloud  and currency: USD
working on BU: Experience Cloud  and currency: USD
working on BU: Print & Publishing  and currency: USD


In [72]:
df_fcst.head(40)
#df_fcst.sample(20)
#df_fcst.tail(20)

,curr,BU,period,recognized_DC,service_DC,deferred_B_DC,deferred_1M_DC,deferred_3M_DC,deferred_6M_DC,deferred_1Y_DC,deferred_2Y_DC,deferred_3Y_DC,Period_Weeks,r_squared,intercept,coeff,X_length
0,ARS,Creative,2020-07,6786.00,0.0,0.0,2.269996e+07,0.0,0.0,953875.89,0.0,0.0,4,0.984194,-6.783323e+06,346064.209423,19.0
1,ARS,Creative,2020-08,16472.00,0.0,0.0,2.408421e+07,0.0,0.0,963180.00,0.0,0.0,4,0.984194,-6.783323e+06,346064.209423,19.0
2,ARS,Creative,2020-09,19205.00,0.0,0.0,3.183559e+07,0.0,0.0,2263704.00,0.0,0.0,5,0.984194,-6.783323e+06,346064.209423,19.0
3,ARS,Creative,2020-10,30382.19,0.0,0.0,2.685273e+07,0.0,0.0,1318346.81,0.0,0.0,4,0.984194,-6.783323e+06,346064.209423,19.0
4,ARS,Creative,2020-11,59049.00,0.0,0.0,2.823698e+07,0.0,0.0,1494162.20,0.0,0.0,4,0.984194,-6.783323e+06,346064.209423,19.0
5,ARS,Creative,2020-12,71355.57,0.0,0.0,3.702655e+07,0.0,0.0,3154860.03,0.0,0.0,5,0.984194,-6.783323e+06,346064.209423,19.0
6,ARS,Creative,2021-01,79224.95,0.0,0.0,3.875687e+07,0.0,0.0,916705.92,0.0,0.0,5,0.984194,-6.783323e+06,346064.209423,19.0
7,ARS,Creative,2021-02,36596.00,0.0,0.0,3.238975e+07,0.0,0.0,1337025.94,0.0,0.0,4,0.984194,-6.783323e+06,346064.209423,19.0
8,ARS,Creative,2021-03,49107.00,0.0,0.0,4.221751e+07,0.0,0.0,2611544.83,0.0,0.0,5,0.984194,-6.783323e+06,346064.209423,19.0
9,ARS,Creative,2021-04,67878.00,0.0,0.0,3.515827e+07,0.0,0.0,3017041.60,0.0,0.0,4,0.984194,-6.783323e+06,346064.209423,19.0


### THIS WOULD BE A GREAT PLACE TO PUT AN INTERACTIVE CHART TO SEE WHAT IS GOING ON

In [73]:
test_output = df_fcst[(df_fcst['curr']=='EUR')&
                     (df_fcst['BU']=='Creative')]
test_output.head(20)

,curr,BU,period,recognized_DC,service_DC,deferred_B_DC,deferred_1M_DC,deferred_3M_DC,deferred_6M_DC,deferred_1Y_DC,deferred_2Y_DC,deferred_3Y_DC,Period_Weeks,r_squared,intercept,coeff,X_length
480,EUR,Creative,2020-07,780757.96,0.0,0.0,4.607758e+07,105013.82,0.0,4.317279e+07,229320.2,0.0,4,0.983894,3.137244e+06,155225.013886,0.0
481,EUR,Creative,2020-08,707326.07,0.0,0.0,4.669848e+07,338.66,0.0,3.965515e+07,392055.8,0.0,4,0.983894,3.137244e+06,155225.013886,0.0
482,EUR,Creative,2020-09,829607.73,0.0,0.0,5.914922e+07,0.00,9110.0,5.254989e+07,284262.7,0.0,5,0.983894,3.137244e+06,155225.013886,0.0
483,EUR,Creative,2020-10,1162722.29,0.0,0.0,4.794028e+07,105013.82,0.0,4.435657e+07,171279.7,0.0,4,0.983894,3.137244e+06,155225.013886,0.0
484,EUR,Creative,2020-11,953689.68,0.0,0.0,4.856118e+07,338.66,0.0,4.619956e+07,920406.7,0.0,4,0.983894,3.137244e+06,155225.013886,0.0
485,EUR,Creative,2020-12,2795172.43,0.0,0.0,6.147760e+07,0.00,0.0,1.058091e+08,511553.2,0.0,5,0.983894,3.137244e+06,155225.013886,0.0
486,EUR,Creative,2021-01,1245220.16,0.0,0.0,6.225372e+07,105013.82,0.0,5.355025e+07,448226.2,0.0,5,0.983894,3.137244e+06,155225.013886,0.0
487,EUR,Creative,2021-02,1167648.01,0.0,0.0,5.042388e+07,338.66,0.0,4.394082e+07,124982.0,0.0,4,0.983894,3.137244e+06,155225.013886,0.0
488,EUR,Creative,2021-03,2088595.39,0.0,0.0,6.380597e+07,0.00,9110.0,5.948547e+07,117158.2,0.0,5,0.983894,3.137244e+06,155225.013886,0.0
489,EUR,Creative,2021-04,933182.00,0.0,0.0,5.166568e+07,105013.82,0.0,4.002136e+07,104344.8,0.0,4,0.983894,3.137244e+06,155225.013886,0.0


In [74]:
df_fcst.columns

Index(['curr', 'BU', 'period', 'recognized_DC', 'service_DC', 'deferred_B_DC',
       'deferred_1M_DC', 'deferred_3M_DC', 'deferred_6M_DC', 'deferred_1Y_DC',
       'deferred_2Y_DC', 'deferred_3Y_DC', 'Period_Weeks', 'r_squared',
       'intercept', 'coeff', 'X_length'],
      dtype='object')

### saving the initial work here

In [75]:
df_billings=df_billings.sort_values(['curr', 'BU', 'period'], ascending = (True, True, True))
df_fcst = df_fcst.sort_values(['curr', 'BU', 'period'], ascending = (True, True, True))

input_df_dict = {'model_dict': model_dict,
                 'billings':df_billings, 
                 'ADBE_cal':df_cal,
                 'bookings': df_book_period,
                 'FX_forwards': df_FX_fwds,
                 'FX_rates': df_FX_rates,
                 'forecast': df_fcst
                }

pickle.dump(input_df_dict, open('../data/processed/initial_forecast.p', 'wb'))

NameError: name 'model_dict' is not defined

In [76]:
df_billings.head(5)

,curr,BU,period,recognized_DC,recognized_US,service_DC,service_US,deferred_B_DC,deferred_B_US,_merge,...,deferred_1Y_DC,deferred_2Y_DC,deferred_3Y_DC,deferred_1M_US,deferred_3M_US,deferred_6M_US,deferred_1Y_US,deferred_2Y_US,deferred_3Y_US,Period_Weeks
0,ARS,Creative,2017-07,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,ARS,Creative,2017-08,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ARS,Creative,2017-09,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,ARS,Creative,2017-10,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,ARS,Creative,2017-11,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
df_fcst.tail(5)

,curr,BU,period,recognized_DC,service_DC,deferred_B_DC,deferred_1M_DC,deferred_3M_DC,deferred_6M_DC,deferred_1Y_DC,deferred_2Y_DC,deferred_3Y_DC,Period_Weeks,r_squared,intercept,coeff,X_length
1615,USD,Print & Publishing,2021-02,6806243.20,0.0,33333.0,304619.272677,200125.00,16456.50,5.634690e+06,1005.35,0.0,4,0.709016,128323.35155,-855.221859,17.0
1616,USD,Print & Publishing,2021-03,6369245.97,0.0,0.0,376497.981553,4792.45,619.00,1.078550e+07,266797.43,0.0,5,0.709016,128323.35155,-855.221859,17.0
1617,USD,Print & Publishing,2021-04,684593.15,0.0,33333.0,297777.497808,108234.18,0.00,6.725653e+06,104787.35,0.0,4,0.709016,128323.35155,-855.221859,17.0
1618,USD,Print & Publishing,2021-05,1656955.97,0.0,0.0,294356.610373,200125.00,2700.00,8.604426e+06,50646.00,0.0,4,0.709016,128323.35155,-855.221859,17.0
1619,USD,Print & Publishing,2021-06,10336697.21,0.0,50000.0,363669.653673,4792.45,77198.53,1.313449e+07,78797.00,0.0,5,0.709016,128323.35155,-855.221859,17.0


In [78]:
df_FX_rates

,DC,Ticker,Spot,FWD_3M,FWD_6M,FWD_9M,FWD_1Y,VOL_3M,VOL_6M,VOL_9M,VOL_1Y
0,ARS,USDARS,68.98350,80.651000,91.571750,104.085600,113.011300,0.258050,0.276550,0.286138,0.295725
1,AUD,AUDUSD,0.69660,0.696592,0.696469,0.696354,0.696128,0.110250,0.108750,0.106463,0.104175
2,BRL,USDBRL,4.96505,4.983550,4.996050,5.006050,5.028784,0.179400,0.167025,0.160562,0.154100
3,CAD,USDCAD,1.34300,1.343012,1.343061,1.342919,1.343325,0.067250,0.066775,0.065862,0.064950
4,CHF,USDCHF,0.96230,0.959683,0.956987,0.953947,0.951229,0.058500,0.061700,0.062300,0.062900
5,CLP,USDCLP,769.15000,769.350000,769.450000,769.300000,769.450000,0.115900,0.116850,0.116650,0.116450
6,COP,USDCOP,3575.07500,3602.200000,3627.575000,3648.700000,3672.775000,0.148525,0.146550,0.142563,0.138575
7,DKK,USDDKK,6.60715,6.593353,6.578650,6.560550,6.543713,0.065575,0.065400,0.065275,0.065150
8,EUR,EURUSD,1.12870,1.131029,1.133346,1.136088,1.138488,0.063875,0.064450,0.063950,0.063450
9,GBP,GBPUSD,1.26705,1.267584,1.268036,1.268750,1.269117,0.085750,0.090850,0.093400,0.095950


##### Creating USD amounts for the forecast
We have the document currency forecast of billings and now have to translate this into USD
 - df_fcst (contains the forecast and historical billings)
 - FX_rates (contains current spot rates, forward rates and volatilities)
 
 ##### I will need to create a 12 month forward vector for each currency
  - First add 'is_direct' field to the df_FX_rates DataFrame

In [79]:
def interp_FX_fwds(df_FX_rates):
    ''' Creates monthly interpolated rates from the df_FX_rates file and adds the is_direct field '''
    # Create list of tickers to determine which is direct (if USD is the first currency, it is direct)
    tickers = df_FX_rates['Ticker'].copy()
    first_curr = [sub[:-3] for sub in tickers]
    is_direct = []
    for curr in first_curr:
        if curr=='USD':
            is_direct.append(0)
        else:
            is_direct.append(1)

    df_FX_rates['is_direct']=is_direct
    
    # Add new columns that will hold the forward rates
    new_cols = ['fwd_01M', 'fwd_02M', 'fwd_03M',
               'fwd_04M', 'fwd_05M', 'fwd_06M',
               'fwd_07M', 'fwd_08M', 'fwd_09M',
               'fwd_10M', 'fwd_11M', 'fwd_12M']

    for item in new_cols:
        df_FX_rates[item]=0
        
    # Interpolate the forward rates
    interp_time = np.arange(1, 13)
    interp_time = interp_time/12

    fwd_times = [0, .25, .5, .75, 1]
    
    for index, row in df_FX_rates.iterrows():
        fwds = [row['Spot'], row['FWD_3M'], row['FWD_6M'], row['FWD_9M'], row['FWD_1Y']]
        interp_fwds = np.interp(interp_time, fwd_times, fwds)
        for i in np.arange(len(new_cols)):
        
            df_FX_rates.loc[index, new_cols[i]]=interp_fwds[i]
    
    return df_FX_rates


In [80]:
df_FX_rates = interp_FX_fwds(df_FX_rates)

In [81]:
df_FX_rates.head(15)

,DC,Ticker,Spot,FWD_3M,FWD_6M,FWD_9M,FWD_1Y,VOL_3M,VOL_6M,VOL_9M,...,fwd_03M,fwd_04M,fwd_05M,fwd_06M,fwd_07M,fwd_08M,fwd_09M,fwd_10M,fwd_11M,fwd_12M
0,ARS,USDARS,68.98350,80.651000,91.571750,104.085600,113.011300,0.258050,0.276550,0.286138,...,80.651000,84.291250,87.931500,91.571750,95.743033,99.914317,104.085600,107.060833,110.036067,113.011300
1,AUD,AUDUSD,0.69660,0.696592,0.696469,0.696354,0.696128,0.110250,0.108750,0.106463,...,0.696592,0.696551,0.696510,0.696469,0.696430,0.696392,0.696354,0.696279,0.696203,0.696128
2,BRL,USDBRL,4.96505,4.983550,4.996050,5.006050,5.028784,0.179400,0.167025,0.160562,...,4.983550,4.987717,4.991883,4.996050,4.999383,5.002717,5.006050,5.013628,5.021206,5.028784
3,CAD,USDCAD,1.34300,1.343012,1.343061,1.342919,1.343325,0.067250,0.066775,0.065862,...,1.343012,1.343028,1.343045,1.343061,1.343014,1.342966,1.342919,1.343054,1.343190,1.343325
4,CHF,USDCHF,0.96230,0.959683,0.956987,0.953947,0.951229,0.058500,0.061700,0.062300,...,0.959683,0.958784,0.957886,0.956987,0.955974,0.954960,0.953947,0.953041,0.952135,0.951229
5,CLP,USDCLP,769.15000,769.350000,769.450000,769.300000,769.450000,0.115900,0.116850,0.116650,...,769.350000,769.383333,769.416667,769.450000,769.400000,769.350000,769.300000,769.350000,769.400000,769.450000
6,COP,USDCOP,3575.07500,3602.200000,3627.575000,3648.700000,3672.775000,0.148525,0.146550,0.142563,...,3602.200000,3610.658333,3619.116667,3627.575000,3634.616667,3641.658333,3648.700000,3656.725000,3664.750000,3672.775000
7,DKK,USDDKK,6.60715,6.593353,6.578650,6.560550,6.543713,0.065575,0.065400,0.065275,...,6.593353,6.588452,6.583551,6.578650,6.572617,6.566583,6.560550,6.554938,6.549325,6.543713
8,EUR,EURUSD,1.12870,1.131029,1.133346,1.136088,1.138488,0.063875,0.064450,0.063950,...,1.131029,1.131801,1.132574,1.133346,1.134260,1.135174,1.136088,1.136888,1.137688,1.138488
9,GBP,GBPUSD,1.26705,1.267584,1.268036,1.268750,1.269117,0.085750,0.090850,0.093400,...,1.267584,1.267735,1.267885,1.268036,1.268274,1.268512,1.268750,1.268872,1.268995,1.269117


In [82]:
df_FX_rates.columns

Index(['DC', 'Ticker', 'Spot', 'FWD_3M', 'FWD_6M', 'FWD_9M', 'FWD_1Y',
       'VOL_3M', 'VOL_6M', 'VOL_9M', 'VOL_1Y', 'is_direct', 'fwd_01M',
       'fwd_02M', 'fwd_03M', 'fwd_04M', 'fwd_05M', 'fwd_06M', 'fwd_07M',
       'fwd_08M', 'fwd_09M', 'fwd_10M', 'fwd_11M', 'fwd_12M'],
      dtype='object')

##### Creating USD forecast
 - loop through the currencies and business units again
 - find the forward rates that need to be calculated, transpose and invert if is_direct = 1
 - take the time index and loop through the forwards to apply the forward rates to each DC amount
 - 
 

# TO BE DONE:
 - create the loop to iterate over each BU and currency (function is already written and used above)
 - multiply each of the following by the tranp_fwds to get the USD amounts
     - recognized_DC
     - service_DC
     - deferred_B_DC
     - deferred_1M_DC
     - deferred_3M_DC
     - deferred_6M_DC
     - deferred_1Y_DC
     - deferred_2Y_DC
     - deferred_3Y_DC
 
 - make this function easy to use by tweaking the forward rates or shocking the FX_rates 
      - This will be a call to the df_FX_rates which will then recalculate the forward rates
      - Then the new forward rates will need to be fed back into the df_fcst dataframe to recalculate USD amounts

In [83]:
new_columns = ['fwd_01M', 'fwd_02M', 'fwd_03M',
               'fwd_04M', 'fwd_05M', 'fwd_06M',
               'fwd_07M', 'fwd_08M', 'fwd_09M',
               'fwd_10M', 'fwd_11M', 'fwd_12M']

In [84]:
list_columns = ['recognized_', 
                'service_',
                'deferred_B_',
                'deferred_1M_',
                'deferred_3M_',
                'deferred_6M_',
                'deferred_1Y_',
                'deferred_2Y_',
                'deferred_3Y_',
               ]

In [85]:
df_fcst.columns

Index(['curr', 'BU', 'period', 'recognized_DC', 'service_DC', 'deferred_B_DC',
       'deferred_1M_DC', 'deferred_3M_DC', 'deferred_6M_DC', 'deferred_1Y_DC',
       'deferred_2Y_DC', 'deferred_3Y_DC', 'Period_Weeks', 'r_squared',
       'intercept', 'coeff', 'X_length'],
      dtype='object')

In [86]:
def convert_fcst(df_fcst, df_FX_rates, list_columns, new_columns):
    
    for i in list_columns:
        new_column = i+'US'
        df_fcst[new_column]= 0
    
    # get the unique list of currency and BU combinations in the forecast
    v_un_BU, v_un_curr = find_unique_curr_and_BU(df_fcst)
    for i in range(len(v_un_BU)):
        this_BU = v_un_BU[i]
        this_curr = v_un_curr[i]
        print('working on BU: {0}  and currency: {1}'.format(this_BU, this_curr))
        
        # create the list of forwards to use here
        these_forwards = df_FX_rates[df_FX_rates['DC']==this_curr]
        just_forwards = these_forwards[new_columns]
        if these_forwards.is_direct.values == 1:
            
            transp_fwds= just_forwards.transpose(copy=True).values
        
        else:
            transp_fwds = just_forwards.transpose(copy=True).values
            transp_fwds = 1/transp_fwds
        
        this_slice = df_fcst[(df_fcst['BU']==this_BU)&
                             (df_fcst['curr']==this_curr)].copy()
        
        for col in list_columns:
            new_column = col+'US'
            old_column = col+'DC'
            
            DC_values =this_slice[old_column].values
            DC_values = DC_values.reshape(-1,1)
            transp_fwds = transp_fwds.reshape(-1,1)
            xx = DC_values * transp_fwds
            
            df_fcst.loc[(df_fcst['BU']==this_BU)&
                           (df_fcst['curr']==this_curr),
                           new_column] = xx

    return df_fcst

In [87]:
df_fcst = convert_fcst(df_fcst, df_FX_rates, list_columns, new_columns)

working on BU: Creative  and currency: ARS
working on BU: DX Other  and currency: ARS
working on BU: Document Cloud  and currency: ARS
working on BU: Experience Cloud  and currency: ARS
working on BU: Print & Publishing  and currency: ARS
working on BU: Creative  and currency: AUD
working on BU: DX Other  and currency: AUD
working on BU: Document Cloud  and currency: AUD
working on BU: Experience Cloud  and currency: AUD
working on BU: Print & Publishing  and currency: AUD
working on BU: Creative  and currency: BRL
working on BU: DX Other  and currency: BRL
working on BU: Document Cloud  and currency: BRL
working on BU: Experience Cloud  and currency: BRL
working on BU: Print & Publishing  and currency: BRL
working on BU: Creative  and currency: CAD
working on BU: DX Other  and currency: CAD
working on BU: Document Cloud  and currency: CAD
working on BU: Experience Cloud  and currency: CAD
working on BU: Print & Publishing  and currency: CAD
working on BU: Creative  and currency: CHF
w

In [88]:
df_fcst.head(40)

,curr,BU,period,recognized_DC,service_DC,deferred_B_DC,deferred_1M_DC,deferred_3M_DC,deferred_6M_DC,deferred_1Y_DC,...,X_length,recognized_US,service_US,deferred_B_US,deferred_1M_US,deferred_3M_US,deferred_6M_US,deferred_1Y_US,deferred_2Y_US,deferred_3Y_US
0,ARS,Creative,2020-07,6786.00,0.0,0.0,2.269996e+07,0.0,0.0,953875.89,...,19.0,93.121335,0.0,0.0,311501.652502,0.0,0.0,13089.625145,0.0,0.0
1,ARS,Creative,2020-08,16472.00,0.0,0.0,2.408421e+07,0.0,0.0,963180.00,...,19.0,214.585808,0.0,0.0,313752.445463,0.0,0.0,12547.641949,0.0,0.0
2,ARS,Creative,2020-09,19205.00,0.0,0.0,3.183559e+07,0.0,0.0,2263704.00,...,19.0,238.124760,0.0,0.0,394732.702698,0.0,0.0,28067.897484,0.0,0.0
3,ARS,Creative,2020-10,30382.19,0.0,0.0,2.685273e+07,0.0,0.0,1318346.81,...,19.0,360.442988,0.0,0.0,318570.748470,0.0,0.0,15640.375602,0.0,0.0
4,ARS,Creative,2020-11,59049.00,0.0,0.0,2.823698e+07,0.0,0.0,1494162.20,...,19.0,671.534092,0.0,0.0,321124.778261,0.0,0.0,16992.342903,0.0,0.0
5,ARS,Creative,2020-12,71355.57,0.0,0.0,3.702655e+07,0.0,0.0,3154860.03,...,19.0,779.231259,0.0,0.0,404344.684323,0.0,0.0,34452.328693,0.0,0.0
6,ARS,Creative,2021-01,79224.95,0.0,0.0,3.875687e+07,0.0,0.0,916705.92,...,19.0,827.474828,0.0,0.0,404800.955688,0.0,0.0,9574.648808,0.0,0.0
7,ARS,Creative,2021-02,36596.00,0.0,0.0,3.238975e+07,0.0,0.0,1337025.94,...,19.0,366.273836,0.0,0.0,324175.303733,0.0,0.0,13381.725308,0.0,0.0
8,ARS,Creative,2021-03,49107.00,0.0,0.0,4.221751e+07,0.0,0.0,2611544.83,...,19.0,471.794369,0.0,0.0,405603.786576,0.0,0.0,25090.356687,0.0,0.0
9,ARS,Creative,2021-04,67878.00,0.0,0.0,3.515827e+07,0.0,0.0,3017041.60,...,19.0,634.013372,0.0,0.0,328395.235993,0.0,0.0,28180.628770,0.0,0.0


###### Checking with a slice of the dataframe df_fcst

In [89]:
us_slice = df_fcst[(df_fcst['BU']=='Creative')&
                  (df_fcst['curr']=='JPY')]
us_slice.head(10)

,curr,BU,period,recognized_DC,service_DC,deferred_B_DC,deferred_1M_DC,deferred_3M_DC,deferred_6M_DC,deferred_1Y_DC,...,X_length,recognized_US,service_US,deferred_B_US,deferred_1M_US,deferred_3M_US,deferred_6M_US,deferred_1Y_US,deferred_2Y_US,deferred_3Y_US
840,JPY,Creative,2020-07,63768286.0,0.0,0.0,2.140959e+09,0.0,0.0,2.832252e+09,...,0.0,5.820783e+05,0.0,0.000000,1.954272e+07,0.0,0.0,2.585286e+07,1518.172521,0.0
841,JPY,Creative,2020-08,89513383.0,0.0,0.0,2.168342e+09,0.0,0.0,3.368465e+09,...,0.0,8.174698e+05,0.0,0.000000,1.980212e+07,0.0,0.0,3.076209e+07,69869.248734,0.0
842,JPY,Creative,2020-09,107644957.0,0.0,400000.0,2.744657e+09,0.0,0.0,4.278443e+09,...,0.0,9.835233e+05,0.0,3654.693586,2.507721e+07,0.0,0.0,3.909100e+07,21274.702302,0.0
843,JPY,Creative,2020-10,70861656.0,0.0,0.0,2.223110e+09,0.0,0.0,3.087487e+09,...,0.0,6.477868e+05,0.0,0.000000,2.032271e+07,0.0,0.0,2.822448e+07,6081.703696,0.0
844,JPY,Creative,2020-11,36111449.0,0.0,0.0,2.250493e+09,0.0,0.0,2.166703e+09,...,0.0,3.302902e+05,0.0,0.000000,2.058394e+07,0.0,0.0,1.981756e+07,13691.079962,0.0
845,JPY,Creative,2020-12,75835728.0,0.0,240000.0,2.847346e+09,0.0,0.0,4.209257e+09,...,0.0,6.939923e+05,0.0,2196.301794,2.605680e+07,0.0,0.0,3.852000e+07,6088.148572,0.0
846,JPY,Creative,2021-01,75510864.0,0.0,0.0,2.881576e+09,0.0,0.0,2.540018e+09,...,0.0,6.915539e+05,0.0,0.000000,2.639044e+07,0.0,0.0,2.326234e+07,91392.873105,0.0
847,JPY,Creative,2021-02,63359981.0,0.0,0.0,2.332645e+09,0.0,0.0,2.333689e+09,...,0.0,5.807213e+05,0.0,0.000000,2.137968e+07,0.0,0.0,2.138926e+07,6034.058726,0.0
848,JPY,Creative,2021-03,155469545.0,0.0,0.0,2.950035e+09,0.0,0.0,4.337474e+09,...,0.0,1.426049e+06,0.0,0.000000,2.705928e+07,0.0,0.0,3.978560e+07,5520.898688,0.0
849,JPY,Creative,2021-04,85910585.0,0.0,1925000.0,2.387412e+09,0.0,0.0,3.277680e+09,...,0.0,7.885095e+05,0.0,17668.147222,2.191228e+07,0.0,0.0,3.008339e+07,6778.400744,0.0


In [90]:
dc = us_slice['deferred_1Y_DC']
us = us_slice['deferred_1Y_US']
print(dc/us)

840    109.552767
841    109.500533
842    109.448300
843    109.390400
844    109.332500
845    109.274600
846    109.190133
847    109.105667
848    109.021200
849    108.953133
850    108.885067
851    108.817000
dtype: float64


In [91]:
df_fcst.columns

Index(['curr', 'BU', 'period', 'recognized_DC', 'service_DC', 'deferred_B_DC',
       'deferred_1M_DC', 'deferred_3M_DC', 'deferred_6M_DC', 'deferred_1Y_DC',
       'deferred_2Y_DC', 'deferred_3Y_DC', 'Period_Weeks', 'r_squared',
       'intercept', 'coeff', 'X_length', 'recognized_US', 'service_US',
       'deferred_B_US', 'deferred_1M_US', 'deferred_3M_US', 'deferred_6M_US',
       'deferred_1Y_US', 'deferred_2Y_US', 'deferred_3Y_US'],
      dtype='object')

In [92]:
df_billings.columns

Index(['curr', 'BU', 'period', 'recognized_DC', 'recognized_US', 'service_DC',
       'service_US', 'deferred_B_DC', 'deferred_B_US', '_merge',
       'deferred_1M_DC', 'deferred_3M_DC', 'deferred_6M_DC', 'deferred_1Y_DC',
       'deferred_2Y_DC', 'deferred_3Y_DC', 'deferred_1M_US', 'deferred_3M_US',
       'deferred_6M_US', 'deferred_1Y_US', 'deferred_2Y_US', 'deferred_3Y_US',
       'Period_Weeks'],
      dtype='object')

### Adding the bookings data to the df_fcst. columns

In [93]:
df_book_period.head(10)

,BU,curr,Q1,Q2,Q3,Q4,P01,P02,P03,P04,...,P03_US,P04_US,P05_US,P06_US,P07_US,P08_US,P09_US,P10_US,P11_US,P12_US
0,Experience Cloud,USD,5.089240e+07,5.512803e+07,6.098239e+07,6.098239e+07,1.837241e+07,1.859049e+07,1.392950e+07,2.631543e+07,...,1.392950e+07,2.631543e+07,1.743999e+07,1.137262e+07,1.492155e+07,1.439104e+07,3.166980e+07,1.893939e+07,1.940309e+07,2.263992e+07
1,Experience Cloud,EUR,4.025977e+06,8.447370e+06,2.220741e+07,2.220741e+07,7.742743e+05,2.235805e+06,1.015897e+06,3.651946e+06,...,9.263407e+05,3.330008e+06,1.713096e+06,2.659588e+06,8.877976e+06,4.582569e+06,6.789170e+06,7.737761e+06,4.602404e+06,7.909550e+06
2,Experience Cloud,GBP,2.846779e+05,3.369984e+06,1.582197e+07,1.582197e+07,7.581794e+04,7.802262e+04,1.308374e+05,1.116178e+06,...,1.069555e+05,9.124407e+05,8.352645e+05,1.007152e+06,6.318615e+06,3.231843e+06,3.383516e+06,4.266901e+06,2.369549e+06,6.297524e+06
3,Experience Cloud,JPY,3.406932e+05,-1.059024e+05,6.017759e+06,6.017759e+06,1.173908e+05,1.431906e+05,8.011178e+04,-4.982438e+04,...,8.635353e+06,-5.370634e+06,-5.425703e+06,-6.190248e+05,2.617557e+08,1.294026e+08,2.575038e+08,1.308845e+08,2.775673e+08,2.402104e+08
4,Experience Cloud,AUD,4.096071e+06,1.039328e+07,7.539179e+06,7.539179e+06,1.451821e+06,1.276282e+06,1.367968e+06,5.342251e+06,...,2.091804e+06,8.169007e+06,4.271145e+06,3.452548e+06,2.095308e+06,3.207033e+06,6.226058e+06,3.541039e+06,1.897216e+06,6.090145e+06
5,Creative,USD,3.108810e+07,2.158878e+07,4.771430e+07,4.771430e+07,1.052358e+07,8.057015e+06,1.250750e+07,7.695929e+06,...,1.250750e+07,7.695929e+06,5.811376e+06,8.081478e+06,1.343259e+07,1.249618e+07,2.178553e+07,1.305227e+07,1.448459e+07,2.017744e+07
6,Creative,EUR,7.196467e+06,7.136804e+06,1.708626e+07,1.708626e+07,2.454970e+06,2.014433e+06,2.727065e+06,2.444050e+06,...,2.486661e+06,2.228595e+06,1.812451e+06,2.466613e+06,4.968561e+06,4.563731e+06,6.047729e+06,3.519341e+06,3.665567e+06,8.395113e+06
7,Creative,GBP,2.163545e+06,1.622120e+06,4.194835e+06,4.194835e+06,7.650463e+05,6.022454e+05,7.962534e+05,5.591267e+05,...,6.509125e+05,4.570687e+05,3.333587e+05,5.356054e+05,9.663864e+05,9.814167e+05,1.481345e+06,1.047405e+06,8.761381e+05,1.505604e+06
8,Creative,JPY,2.466702e+06,5.101532e+06,6.046827e+06,6.046827e+06,6.802024e+05,6.249488e+05,1.161551e+06,1.574610e+06,...,1.252051e+08,1.697292e+08,1.796841e+08,2.004875e+08,1.761638e+08,2.095158e+08,2.661157e+08,2.126508e+08,1.492318e+08,2.899128e+08
9,Creative,AUD,1.532729e+06,1.636074e+06,3.460680e+06,3.460680e+06,5.067057e+05,3.799703e+05,6.460531e+05,5.223274e+05,...,9.879005e+05,7.987075e+05,7.060838e+05,9.969823e+05,1.598439e+06,1.562617e+06,2.130779e+06,1.358659e+06,1.172276e+06,2.760901e+06


#### Need to take each BU/curr combination in the df_book_period rows and pull P07, P08, P09 ... P12 and P07_US, P08_US ... P12_US and move them to the df_fcst dataframe under the correct BU / curr / period section

In [99]:
def merge_bookings_to_fcst(df_book_period, df_fcst):
    dc_list = ['P07', 'P08', 'P09',
               'P10', 'P11', 'P12']
    us_list = ['P07_US', 'P08_US', 'P09_US',
               'P10_US', 'P11_US', 'P12_US']


    df_temp_book_period = df_book_period.drop(columns=['Q1', 'Q2', 'Q3', 'Q4',
                                                     'P01', 'P02', 'P03', 'P04', 'P05', 'P06',
                                                     'P01_US', 'P02_US', 'P03_US',
                                                      'P04_US', 'P05_US', 'P06_US', 'FX_fwd_rate']).copy()

    df_DC = df_temp_book_period.copy()
    df_US = df_temp_book_period.copy()

    df_DC = df_DC.drop(columns = us_list)
    df_US = df_US.drop(columns = dc_list)

    df_DC_melt = pd.melt(df_DC, id_vars = ['BU', 'curr'])
    df_US_melt = pd.melt(df_US, id_vars = ['BU', 'curr'])

    df_DC_melt.rename(columns={'variable': 'period',               
                              'value': 'book_1Y_DC'}, inplace=True)
    df_US_melt.rename(columns={'variable': 'period',
                              'value': 'book_1Y_US'}, inplace=True)

    df_DC_melt['period'] = df_DC_melt['period'].str.replace('P', '2020-')
    df_US_melt['period'] = df_US_melt['period'].str.replace('P', '2020-')
    df_US_melt['period'] = df_US_melt['period'].str.replace('_US', '')


    # reset index
    df_DC_melt.set_index(['BU', 'curr', 'period'], inplace=True)

    df_US_melt.set_index(['BU', 'curr', 'period'], inplace=True)

    df_melted = df_DC_melt.join(df_US_melt, how='left')   

    df_fcst.set_index(['BU', 'curr', 'period'], inplace=True)
    
    df_fcst = df_fcst.join(df_melted, how='left')
    df_fcst.fillna(0, inplace=True)
    df_fcst.reset_index(inplace=True)
    return df_fcst

In [100]:
df_fcst = merge_bookings_to_fcst(df_book_period, df_fcst)

KeyError: "None of ['BU', 'curr', 'period'] are in the columns"

In [98]:
df_fcst.head(20)

recognized_DC  service_DC  deferred_B_DC  \
BU       curr period                                              
Creative ARS  2020-07        6786.00         0.0            0.0   
              2020-08       16472.00         0.0            0.0   
              2020-09       19205.00         0.0            0.0   
              2020-10       30382.19         0.0            0.0   
              2020-11       59049.00         0.0            0.0   
              2020-12       71355.57         0.0            0.0   
              2021-01       79224.95         0.0            0.0   
              2021-02       36596.00         0.0            0.0   
              2021-03       49107.00         0.0            0.0   
              2021-04       67878.00         0.0            0.0   
              2021-05      141104.00         0.0            0.0   
              2021-06      127456.19         0.0            0.0   
DX Other ARS  2020-07           0.00         0.0            0.0   
              2020-08           0.00         0.0            0.0   
              2020-09           0.00         0.0            0.0   
              2020-10           0.00         0.0            0.0   
              2020-11           0.00         0.0            0.0   
              2020-12           0.00         0.0            0.0   
              2021-01           0.00         0.0            0.0   
              2021-02           0.00         0.0            0.0   

                       deferred_1M_DC  deferred_3M_DC  deferred_6M_DC  \
BU       curr period                                                    
Creative ARS  2020-07    2.269996e+07             0.0             0.0   
              2020-08    2.408421e+07             0.0             0.0   
              2020-09    3.183559e+07             0.0             0.0   
              2020-10    2.685273e+07             0.0             0.0   
              2020-11    2.823698e+07             0.0             0.0   
              2020-12    3.702655e+07             0.0             0.0   
              2021-01    3.875687e+07             0.0             0.0   
              2021-02    3.238975e+07             0.0             0.0   
              2021-03    4.221751e+07             0.0             0.0   
              2021-04    3.515827e+07             0.0             0.0   
              2021-05    3.654252e+07             0.0             0.0   
              2021-06    4.740848e+07             0.0             0.0   
DX Other ARS  2020-07    0.000000e+00             0.0             0.0   
              2020-08    0.000000e+00             0.0             0.0   
              2020-09    0.000000e+00             0.0             0.0   
              2020-10    0.000000e+00             0.0             0.0   
              2020-11    0.000000e+00             0.0             0.0   
              2020-12    0.000000e+00             0.0             0.0   
              2021-01    0.000000e+00             0.0             0.0   
              2021-02    0.000000e+00             0.0             0.0   

                       deferred_1Y_DC  deferred_2Y_DC  deferred_3Y_DC  \
BU       curr period                                                    
Creative ARS  2020-07       953875.89             0.0             0.0   
              2020-08       963180.00             0.0             0.0   
              2020-09      2263704.00             0.0             0.0   
              2020-10      1318346.81             0.0             0.0   
              2020-11      1494162.20             0.0             0.0   
              2020-12      3154860.03             0.0             0.0   
              2021-01       916705.92             0.0             0.0   
              2021-02      1337025.94             0.0             0.0   
              2021-03      2611544.83             0.0             0.0   
              2021-04      3017041.60             0.0             0.0   
              2021-05      2728280.63             0.0            

##### Merging the df_fcst with the df_bililngs for easier charting?

In [ ]:
df_billings['is_forecast']= 0
df_fcst['is_forecast']=1
df = pd.concat([df_billings, df_fcst],
            join='outer',
            ignore_index=True)
df = df.fillna(0)
df.sort_values(by=['curr', 'BU', 'period'], inplace=True)

input_df_dict_short = {'model_dict': model_dict,
                       'ADBE_cal':df_cal,
                       'bookings': df_bookings,
                       'FX_forwards': df_FX_fwds,
                       'FX_rates': df_FX_rates,
                       'final': df
                }
pickle.dump(input_df_dict, open('../data/processed/final_forecast.p', 'wb'))